<a href="https://colab.research.google.com/github/LashawnFofung/AI-Portfolio/blob/main/src/AI_Powered_Document_Intelligence_Automation_Platform_MVP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AI-Powered Document Intelligence Automation Platform MVP**

This notebook serves as a production-ready MVP for a Multi-Modal Document Intelligence system. It is designed to solve complex business problems associated with high-volume document reviews, such as mortgage processing, legal discovery, and HR automation. Unlike standard RAG (Retrieval-Augmented Generation) systems that often suffer from "Context Contamination," this platform utilizes Intelligent Document Boundary Detection and Metadata-Rich Chunking to ensure that queries are routed to the precise document context required.

<br><br>

## **🛠️ MVP Objectives**
- **Contextual Fidelity:** Eliminate "hallucinations" caused by mixing data from different documents within the same vector space.

- **Intelligent Automation:** Automate the classification and segmentation of bulk-uploaded files (e.g., a single PDF containing a resume, an ID, and a contract).

- **Multi-Model Versatility:** Allow users to switch between different LLM engines (Gemini, Mistral, Phi-2) based on performance and latency needs.

- **Production Readiness:** Integrate OCR (Tesseract), vector search (FAISS), and professional PDF report generation into a unified workflow.

<br><br>

## **🏗️ Key Technical Architecture**
The platform is built on a modular pipeline that ensures data integrity from ingestion to output:

- **Ingestion Layer:** Supports PDF and Image files with Hybrid OCR (PyMuPDF + Tesseract) to handle scanned documents.
- **Intelligence Layer:** Uses LLM-driven classification to identify document types and detect logical boundaries between pages.
- **Storage Layer:** Implements Metadata-Rich Chunking using LlamaIndex and FAISS, creating segregated vector indices for each document type to prevent cross-contamination.
- **Orchestration Layer:** A central router predicts the most likely document type for a given query and routes the request to the specific vector index.

<br><br>

## **🌟 Core MVP Capabilities**
- **Multi-Modal Routing:** Automatically detects if a query is about an "Invoice" vs. a "Mortgage Contract" and searches only the relevant documents.
- **Logical Document Segmentation:** Breaks down large, combined PDF files into individual logical units (e.g., separating Page 1-3 as a "Resume" and Page 4 as an "ID").
- **Performance Auditing:** Tracks ROUGE scores and response times to evaluate the quality of the AI's answers.
- **Interactive UI:** A Gradio-based interface featuring a PDF viewer, audit logs, and a model-switching dashboard.

<br><br>

## **📖 How to Operate**
- **Environment Setup:** Run Section 1 and 2 to install dependencies and configure your Gemini API Key in the Colab Secrets.
- **Initialization:** Run the global configuration cells to load the BGE embedding model and initialize the default LLM.
- **Select Engine:** Use the switch_llm function or the UI dropdown to select your preferred AI model (e.g., "Gemini 2.0 Flash").
- **Upload & Process:** Upload your documents via the Gradio interface. The system will automatically classify and index them.
- **Query & Audit:** Enter your business questions. Use the "Audit Log" tab to view performance metrics and download a professional PDF summary of the session.

<br><br>
## **🔍 Section Logic & Flow Analysis**

- **1-2: Setup & Config**
  - Establishes the environment, mounts Google Drive, and initializes global state variables (audit_logs, current_llm).
      - [Section 1: Setup And Installation](#scrollTo=OnjSSFKJmRQc)
      - [Section 2A: Core Imports, Security Keys, And Global Settings Configurations](#scrollTo=1TlPkZLbvExS)
      - [Section 2B: LLM Factory & Resource Configuration](#scrollTo=1MYP_DZAw6QJ)

- **3	Data Structures**
  - Defines PageInfo, LogicalDocument, and ChunkMetadata dataclasses to maintain data schema consistency across the pipeline.
      - [Section 3: Data Structures For Enhanced Document Management](#scrollTo=RLrZv_r30NBc)

- **4-5	Ingestion & OCR**
  - The "Aware Router" (extract_and_analyze_file) directs files to PDF or Image processors. Logic includes LLM-based boundary detection.
      - [Section 4: Document Intelligence Functions](#scrollTo=xQNJUFwB2gMZ)
      - [Section 5: Advanced PDF Processing Pipeline](#scrollTo=yGrQwXDp4gh_)

- **6	Intelligent Chunking**
  - Converts logical documents into overlapping segments while embedding rich metadata (page numbers, doc IDs) into every chunk.
      - [Section 6: Intelligent Chunking With Metadata Preservation](#scrollTo=qYroZgDG9FxX)

- **7-8	Vector Search**
  - Builds segregated FAISS indices. The IntelligentRetriever applies query routing to search only the relevant document "silo".
      - [Section 7: Query Routing And Intelligent Retrieval](#scrollTo=d_MsgfAaAGsm)
      - [Section 8: Enhanced Answer Generation With Source Attribution](#scrollTo=VrLmw0oyEGfq)

- **9-10	Orchestrator**
  - The "Brain" of the platform. It handles the full RAG cycle: Query -> Route -> Retrieve -> Generate -> Audit.
      - [Section 9: Enhanced Document Store](#scrollTo=mJTVS2CcHBq3)
      - [Section 10: Backend Chat & Audit Loogic](#scrollTo=txaCA0n9MNYN)

- **11-12	UI & Reporting**
  - The Gradio interface layer and the ReportLab logic for generating production-ready audit reports.
      - [Section 11:Chatbot Logic & Orchestration](#scrollTo=80rvI0tlTlEu)
      - [Section 12: Gradio Interface, Chat Handlers, & Wiring Logic](#scrollTo=2pYWo7wCa47E)

- **13 [Application Launcher](#scrollTo=GgDAO0pTcury)**





# **SECTION 1. SETUP AND INSTALLATION**

**Logic and Flow Analysis**
This section serves as the Foundation Layer of the AI-Powered Document Intelligence Platform. The logic follows a linear, non-destructive sequence:
1. **Dependency Provisioning:** Installs the multi-modal stack required for the MVP. This includes UI components (`Gradio`), document parsing (`PyMuPDF`, `LlamaIndex`), OCR engines (`Tesseract`), and the vector search backend (FAISS).
2. **Global State Initialization:** Sets up persistent tracking variables (`audit_logs` and `current_llm`). This is a critical design choice for the MVP, as it allows for performance metrics to persist across multiple document uploads and ensures the system knows which LLM engine is currently "warm" in memory.
3. **Asynchronous Handling:** Inspects the event loop to prevent "loop already running" errors common in Jupyter environments when initializing asynchronous RAG pipelines.
4. **Resource Mounting:** Links Google Drive to ensure the UI has access to static assets (logos/branding) and persistent storage for output reports.


In [ ]:
# ------- SECTION 1. SETUP AND INSTALLATION -------


# 1.1 UI, PDF Processing & Machine Learning Foundations
# Grouping core utilities for document extraction and interface building
!pip install -q \
    gradio gradio_pdf \
    pypdf PyPDF2 pymupdf \
    pillow \
    sentence-transformers transformers \
    faiss-cpu \
    google-generativeai \
    numpy pandas jedi\
    json-repair

# # 1.2 LlamaIndex Orchestration Stack
# Specifically for RAG (Retrieval-Augmented Generation) and metadata management
!pip install -q \
    llama-index \
    llama-index-readers-file \
    llama-index-vector-stores-faiss

#1.3 LLM Engine Support (Multi-Modal Switching)
# Libraries required to swap between API-based (Gemini) and Local (HuggingFace) models
!pip install -q \
    llama-index-llms-google-genai \
    llama-index-llms-huggingface \
    llama-index-embeddings-huggingface \
    transformers accelerate bitsandbytes

# 1.4 OCR & Specialized Reporting Tools
# Tesseract for scanned docs; ReportLab for automated PDF performance summaries
!apt-get install -y tesseract-ocr
!pip install -q \
    pytesseract \
    reportlab rouge-score \
    matplotlib seaborn

# --- MISTRAL MDEL INSTALLATION with COLAB GPU  ---
# Install llama-cpp-python with CUDA support for the T4 GPU
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python
# Install the LlamaIndex connector for LlamaCPP and json-repair for Mistral cleaning
!pip install llama-index-llms-llama-cpp json-repair


# --- [GLOBAL STATE INITIALIZATION] ---

# --- 1. Initialize GLOBAL AUDIT LOG for Performance Tracking ---
# audit_logs: Stores performance data (latencies, ROUGE scores) for the report generator
audit_logs = []
print("✅ Global audit_logs list initialized.")

# --- 2. Initialize GLOBAL STATE TRACKING FOR LLM CHOICE ---
# current_llm/name: Tracks the active engine to prevent unnecessary re-loading of weights
current_llm = None
current_model_name = ""
print("✅ Global state for LLM variables initialized.")


# --- [ASYNC & ENVIRONMENT PREP] ---
try:
    loop = asyncio.get_event_loop()
    if loop.is_running():
        print("💡 Event loop is active; preparing for asynchronous RAG operations.")
except:
    pass

# --- [EXTERNAL STORAGE LINKING] ---
# MOUNT GOOGLE DRIVE (For UI Image) ---
from google.colab import drive
drive.mount('/content/drive')

print("✅ SECTION 1. SETUP AND INSTALLATION COMPLETE.")



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
✅ Global audit_logs list initialized.
✅ Global state for LLM variables initialized.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ SECTION 1. SETUP AND INSTALLATION COMPLETE.


# **SECTION 2A. CORE IMPORTS, SECURITY KEYS, AND GLOBAL SETTINGS CONFIGURATIONS**

This section serves as the Operational Command Center for the platform. It transitions the environment from a collection of installed packages to a configured, functional system.

The logic flow is as follows:

1. **Library Orchestration:** Imports are logically categorized. By grouping them, we separate the "Engine" (AI/LLMs) from the "Interface" (Gradio/Reporting) and the "Mechanics" (Document Parsing/OCR).

2. **Resource Verification:** Instead of waiting for the UI to fail, the code proactively validates file paths for essential brand assets (Logos) and architectural diagrams. This is a best practice for production MVPs to ensure the user experience is consistent.

3. **Security & Authentication:** Retrieves the Gemini API key from Colab’s secure userdata (Secrets). This ensures that sensitive keys are never hard-coded into the notebook.

4. **Global RAG Settings:** This is the most critical logic step. By assigning Settings.embed_model and Settings.llm, you establish a "Global Context" within LlamaIndex. This allows every subsequent component (Indexers, Retrievers, and Query Engines) to automatically inherit these configurations without redundant code.


In [ ]:
# ------- SECTION 2A. CORE IMPORTS, SECURITY KEYS, AND GLOBAL SETTINGS CONFIGURATIONS -------

# 1. Standard Library & Utilities
import os, time, json, re, io, tempfile, hashlib, asyncio
import random # Used for simulating performance audit metrics
import gc # Memory Management: Essential for Google Colab T4 GPU
from datetime import datetime
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

# 2. Data Science & Visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# 3. Document Processing & OCR
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
from PyPDF2 import PdfReader


# 4. AI & Machine Learning (Vector Engine/Backend)
# Core Frameworks
import torch # Memory Management: Essential for Google Colab T4 GPU
import faiss
from rouge_score import rouge_scorer
from transformers import BitsAndBytesConfig
from sentence_transformers import SentenceTransformer


# 5. LlamaIndex (RAG Framework)
# The Orchestrator
from llama_index.core.schema import TextNode
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Document, VectorStoreIndex, StorageContext, Settings
from llama_index.core.vector_stores import MetadataFilters, MetadataFilter, FilterOperator

        # --- LLM & Embedding -----
from json_repair import repair_json  # Critical imports for your Mistral/Router logic
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


# 6. UI & Automated PDF Reporting
import gradio as gr
from gradio_pdf import PDF
from reportlab.lib import colors
from google.colab import userdata
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.colors import HexColor, black, green
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle


# --- --- --- [MEMORY MANAGEMENT] --- --- ---
# Shared 4-bit configuration for T4 GPU efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

# --- --- --- [RESOURCE PATH DEFINITIONS] ---- --- ---
# 1A. Define File Path - LOGO
PROJECT_FOLDER = '/content/drive/MyDrive/AI_Powered_Document_Intelligence_Automation_Platform'
LOGO_PATH = os.path.join(PROJECT_FOLDER, 'AI Document Assistant logo v2.png')

# 2A. Define File Path - CONFIG & FILTER IMAGE
CONFIG_FILTER_PATH = os.path.join(PROJECT_FOLDER, 'Document Filter and RAG.png')

# 1B. Verify the path exists to avoid "File Not Found" errors later
if os.path.exists(LOGO_PATH):
    print(f"✅ Image found at: {LOGO_PATH}")
else:
    print(f"❌ Warning: Image not found. Check path: {LOGO_PATH}")


# 2B. Verify the path exists to avoid "File Not Found" errors later
if os.path.exists(CONFIG_FILTER_PATH):
    print(f"✅ Image found at: {CONFIG_FILTER_PATH}")
else:
    print(f"❌ Warning: Image not found. Check path: {CONFIG_FILTER_PATH}")


# --- --- --- [SECURITY & GLOBAL CONFIGURATION] --- --- ---
# 1A. Load Gemini API
API_KEY = userdata.get('GEMINI_API_KEY')
if not API_KEY:
    raise ValueError("GEMINI_API_KEY not found in Colab Secrets.")


# 1B Load Hugginf Face Token
# Retrieve the secret from Colab and set it as an environment variable
try:
    hf_token = userdata.get('HF_TOKEN')
    os.environ["HF_TOKEN"] = hf_token
    print("✅ Hugging Face Token successfully loaded from Colab Secrets.")
except Exception as e:
    print("❌ Could not find HF_TOKEN in Colab Secrets. Check the 'Key' icon on the left.")


# 2. Configure Embedding Model
llama_embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# 3A. GLOBAL CONFIGURATION (Crucial for Section 9 & 10)
Settings.embed_model = llama_embed_model
# 3B. Initial default LLM
Settings.llm = GoogleGenAI(
    model="models/gemini-2.0-flash",
    api_key=API_KEY
)

# 3C. SAFE NAME ASSIGNMENT
# We use a custom attribute that Pydantic won't block,
# or simply use the existing 'model' attribute.
# To satisfy your Section 11 logs, we use this "monkeypatch" method:
try:
    # This bypasses Pydantic's strict check
    object.__setattr__(Settings.llm, 'model_name', "Gemini 2.0 Flash")
except Exception:
    pass

print(f"✅ Settings initialized with: {getattr(Settings.llm, 'model_name', Settings.llm.model)}")


print("✅ SECTION 2A. CORE IMPORTS, SECURITY KEYS LOADED, AND GLOBAL SETTINGS CONFIGURATIONS COMPLETE.")



✅ Image found at: /content/drive/MyDrive/AI_Powered_Document_Intelligence_Automation_Platform/AI Document Assistant logo v2.png
✅ Image found at: /content/drive/MyDrive/AI_Powered_Document_Intelligence_Automation_Platform/Document Filter and RAG.png
✅ Hugging Face Token successfully loaded from Colab Secrets.
✅ Settings initialized with: Gemini 2.0 Flash
✅ SECTION 2A. CORE IMPORTS, SECURITY KEYS LOADED, AND GLOBAL SETTINGS CONFIGURATIONS COMPLETE.


# **SECTION 2B. LLM FACTORY & RESOURCE CONFIGURATION**

**Logic and Flow Analysis**
This section establishes the **Model Agnostic Architecture** of the platform. By decoupling model definition from model execution, the system can seamlessly transition between high-performance cloud APIs (Gemini) and specialized local models (Mistral, Phi-2) without code duplication. This flexibility allows for rapid prototyping and cost-effective scaling within restricted hardware environments.

<br>

The logic flow follows a **"Modular Factory Pattern"**:
1. **Encapsulated Initializers** Individual helper functions serve as "factories" for specific LLM types. This isolates technical complexities—such as **4-bit quantization configs**, device maps, and specific tokenizer paths—away from the core application logic. This modularity ensures that adding a new model only requires creating a new factory function rather than rewriting the orchestration layer.

2. **T4 GPU Optimization (Quantization):** Local models are configured using `BitsAndBytesConfig`. This ensures large models like Mistral 7B are compressed into 4-bit precision, significantly reducing their VRAM footprint from ~15GB to ~5.5GB. This optimization is what makes multi-model experimentation possible on a standard T4 instance without sacrificing significant reasoning capability.

3. **Stateless Definition:** These functions define how to build the "brain" of the AI but do not activate it immediately. This "on-demand" approach is a critical memory management strategy; it prevents the system from attempting to load all models into VRAM simultaneously, which would cause an immediate `CUDA out of memory` error.

In [ ]:
# ------- SECTION 2B. LLM FACTORY & RESOURCE CONFIGURATION -------

# --- 1. MODEL INITIALIZERS ---

### 🧠 Helper function to set up Gemini (External API) ###
def setup_gemini_llm():
    """Initializes the latest Google GenAI model for the platform."""
    # Ensure you use the correct class name GoogleGenAI
    llm = GoogleGenAI(
        model="models/gemini-2.0-flash",
        api_key=API_KEY
    )
    # Pydantic-safe name pinning
    object.__setattr__(llm, 'model_name', "Gemini 2.0 Flash")

    Print("✅ Gemini API Key Loaded & Configured.")

    return llm


### 🧠 Helper function to set up Mistral (External API) ###
def setup_mistral_7b_llm():
    """Downloads and loads Mistral-7B via GGUF to fit ~4.1GB VRAM."""

    model_url = "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf"
    model_path = "/content/mistral-7b-v0.2.Q4_K_M.gguf"

    # Automated check to ensure the file exists
    if not os.path.exists(model_path):
        print("📥 Downloading Mistral (4.1GB)...")
        os.system(f'wget -q {model_url} -O {model_path}')
        print("✅ Download Complete.")

    llm = LlamaCPP(
        model_path=model_path,
        temperature=0.1,
        max_new_tokens=1024,
        context_window=16384, # increase from 8192 to 16384 due to CPU limitations; model was trained to "remember" up to 32,768 tokens
        model_kwargs={"n_gpu_layers": 33}, # Offload 33 layers (full model) to T4 GPU
        messages_to_prompt=lambda messages: "\n".join([f"{m.role}: {m.content}" for m in messages]),
        completion_to_prompt=lambda completion: f"AI: {completion}",
        verbose=False
    )

    # Pydantic-safe name pinning
    object.__setattr__(llm, 'model_name', "Mistral 7B")


    print("✅ Mistral 7B API Key Loaded & Configured.")

    return llm



### 🧠 Helper function to set up Microsoft Phi-2(External API) ###
def setup_phi2_llm():
    """Loads Microsoft Phi-2 (2.7B) - Very fast on T4."""
    llm = HuggingFaceLLM(
        model_name="microsoft/phi-2",
        model_kwargs={"quantization_config": bnb_config, "trust_remote_code": True},
        device_map="auto"
    )

    # Pydantic-safe name pinning
    object.__setattr__(llm, 'model_name', "Microsoft Phi-2")

    print("✅ Phi-2 API Key Loaded & Configured.")

    return llm



### 🧠 Helper function to set up TinyLlama (External API) ###
def setup_tinyllama_llm():
    """Loads TinyLlama (1.1B) - Lowest memory footprint (~2GB)."""
    llm = HuggingFaceLLM(
        model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
        model_kwargs={"quantization_config": bnb_config},
        device_map="auto"
    )
    # Pydantic-safe name pinning
    object.__setattr__(llm, 'model_name', "TinyLlama 1.1B")

    print("✅ TinyLlama API Key Loaded & Configured.")

    return llm

print("✅ SECTION 2B. LLM FACTORY & RESOURCE CONFIGURATION COMPLETE.")



✅ SECTION 2B. LLM FACTORY & RESOURCE CONFIGURATION COMPLETE.


# **SECTION 3. DATA STRUCTURES FOR ENHANCED DOCUMENT MANAGEMENT**

Logic and Flow Analysis
This section defines the **Data Blueprint** for the entire platform. While many RAG systems treat text as "flat strings," this MVP uses Python dataclasses to create a hierarchical representation of a document.

<br>

The logic flows through three granular levels:
1. **Page Level (**`PageInfo`**):** The smallest physical unit. It captures raw text while tagging it with a page number, ensuring that when the AI answers a question, it can cite the exact location in the original PDF.

2. **Logical Level (**`LogicalDocument`**):** The business unit. Often, a single 100-page PDF contains multiple distinct documents (e.g., an Invoice followed by a Contract). This structure allows the system to "boundary-detect" where one document ends and another begins, preventing data leakage between unrelated sections.

3. **Search Level (**`ChunkMetadata`**):** The retrieval unit. This stores the text along with "Rich Metadata" (IDs, types, and page ranges). By including the embedding field directly in the object, the platform can easily pass these structures between the FAISS vector store and the LLM generator.

In [ ]:
# ------- SECTION 3. DATA STRUCTURES FOR ENHANCED DOCUMENT MANAGEMENT -------
@dataclass
class PageInfo:
    """
    PHYSICAL LAYER: Represents one page of the input file.
    Used for OCR tracking and initial classification. Stores information about a single page.
    """
    page_num: int
    text: str
    doc_type: Optional[str] = None
    page_in_doc: int = 0   # Position relative to the logical start

@dataclass
class LogicalDocument:
    """
    BUSINESS LAYER: Groups pages into a single 'semantic' entity.
    Logic: If a PDF has 10 pages, pages 1-3 might be a 'Purchase Order'
    and 4-10 a 'Master Service Agreement'.
    Represents a logical document within a PDF.
    """
    doc_id: str
    doc_type: str
    page_start: int
    page_end: int
    text: str
    chunks: List[Dict] = None

@dataclass
class ChunkMetadata:
    """
    RETRIEVAL LAYER: The actual object indexed in the Vector Database.
    Rich metadata here allows for 'Siloed Retrieval' (filtering by doc_type).
    Rich metadata for each chunk.
    """
    chunk_id: str
    doc_id: str
    doc_type: str
    chunk_index: int
    page_start: int
    page_end: int
    text: str
    embedding: Optional[np.ndarray] = None

print("✅ DATA STRUCTURES INITIALIZED.")

print("✅ SECTION 3. DATA STRUCTURES FOR ENHANCED DOCUMENT MANAGEMENT COMPLETE.")


✅ DATA STRUCTURES INITIALIZED.
✅ SECTION 3. DATA STRUCTURES FOR ENHANCED DOCUMENT MANAGEMENT COMPLETE.


# **SECTION 4. DOCUMENT INTELLIGENCE FUNCTIONS**

**Logic and Flow Analysis**

This section acts as the "Triage Layer" of the platform. It moves beyond simple text extraction and applies semantic intelligence to understand what the document is and where it ends.

<br>

The logic flow consists of two primary cognitive tasks:
1. **Semantic Classification (**`classify_document_type` **):** Instead of using brittle keyword matching, this function utilizes the LLM to understand the context of the page. It maps raw text to a predefined taxonomy (Invoices, Land Deeds, etc.). This categorization is vital for "Siloed Retrieval" later in the pipeline, ensuring that a query about a "loan fee" doesn't accidentally pull data from a "Medical Report."
2. **Logical Boundary Detection
(** `detect_document_boundary` **):** This is the solution to the "Combined PDF" problem. In business workflows, users often scan multiple documents into a single file. This function performs a "bridge analysis" between the end of one page and the start of the next. It checks for continuity in formatting, topic, and sentence structure to decide if the system should start a new `LogicalDocument` object.

In [ ]:
# ------- SECTION 4. DOCUMENT INTELLIGENCE FUNCTIONS -------
def classify_document_type(text: str, max_length: int = 1500) -> str:
    """
    Identifies the document category using semantic analysis.
    Essential for routing queries to the correct document 'silo'.
    """
    # Truncate text if too long to avoid token limits
    # Safety Check: Use a sample to stay within LLM context limits and reduce latency
    text_sample = text[:max_length] if len(text) > max_length else text

    prompt = f"""
    Analyze this document and classify it into ONE of these categories:
    - Resume: CV, professional profile, work history
    - Contract: Legal agreement, terms and conditions, service agreement
    - Mortgage Contract: Home loan agreement, mortgage terms, property financing
    - Invoice: Bill, payment request, financial statement
    - Pay Slip: Salary statement, wage slip, earnings statement
    - Lender Fee Sheet: Loan fees, lender charges, closing costs
    - Land Deed: Property deed, title document, ownership certificate
    - Bank Statement: Account statement, transaction history
    - Tax Document: W2, 1099, tax return, tax form
    - Insurance: Insurance policy, coverage document
    - Report: Analysis, research document, findings
    - Letter: Correspondence, memo, communication
    - Form: Application, questionnaire, data entry form
    - ID Document: Driver's license, passport, identification
    - Medical: Medical report, prescription, health record
    - Other: Doesn't fit other categories

    Document sample:
    {text_sample}

    Respond with ONLY the category name, nothing else.
    """

    try:
        # Use the global LlamaIndex LLM setting
        response = Settings.llm.complete(prompt)
        doc_type = response.text.strip()

        # Normalize the response
        valid_types = [
            'Resume', 'Contract', 'Mortgage Contract', 'Invoice', 'Pay Slip',
            'Lender Fee Sheet', 'Land Deed', 'Bank Statement', 'Tax Document',
            'Insurance', 'Report', 'Letter', 'Form', 'ID Document',
            'Medical', 'Other'
        ]

        # Find best match (case-insensitive)
        for valid_type in valid_types:
            if doc_type.lower() == valid_type.lower():
                return valid_type

        return 'Other'
    except Exception as e:
        print(f"Classification error: {e}")
        return 'Other'

def detect_document_boundary(prev_text: str, curr_text: str,
                            current_doc_type: str = None) -> bool:
    """
    Detect if two consecutive pages belong to the same document.
    Returns True if they're from the same document.
    """
    # Quick heuristic checks first
    if not prev_text or not curr_text:
        return False

    # Sample the texts for L\LM analysis
    prev_sample = prev_text[-500:] if len(prev_text) > 500 else prev_text
    curr_sample = curr_text[:500] if len(curr_text) > 500 else curr_text

    prompt = f"""
    Determine if these two pages are from the SAME document or different documents.

    Current document type: {current_doc_type or 'Unknown'}

    End of Previous Page:
    ...{prev_sample}

    Start of Current Page:
    {curr_sample}...

    Consider:
    - Continuity of content
    - Formatting consistency
    - Topic coherence
    - Page numbers or headers

    Default to 'Yes' unless you see a clear signal of a different entity
    (e.g., a new person's name on a resume, a different bank logo, or a new header 'Exhibit A').
    Answer ONLY 'Yes' or 'No'.
    """

    try:
        # Use the global LlamaIndex LLM setting
        response = Settings.llm.complete(prompt)

        return response.text.strip().lower().startswith('yes')
    except Exception as e:
        print(f"Boundary detection error: {e}")
        # Default to keeping pages together if uncertain
        return True

print("✅ SECTION 4. DOCUMENT INTELLIGENCE FUNCTIONS COMPLETE.")


✅ SECTION 4. DOCUMENT INTELLIGENCE FUNCTIONS COMPLETE.


# **SECTION 5. ADVANCED PDF PROCESSING PIPELINE**


**Logic and Flow Analysis**

This section serves as the **Data Gateway** of the platform. It is designed to handle "Real-World" documents which are often messy, scanned, or combined.

<br>

The logic flow implements a **Smart Routing and Hybrid OCR** strategy:
1. **Aware Router (** `extract_and_analyze_file`**):** Acts as a traffic controller, detecting file extensions and directing the payload to either the PDF or Image processor.

2. **Hybrid OCR Engine (** `extract_and_analyze_pdf` **):** Solves the "Blank Page" problem. If standard text extraction (PyMuPDF) returns an empty string—common in scanned mortgage or legal docs—the system automatically triggers a high-resolution render and passes it to **Tesseract OCR**.

3. **Boundary Integration (** `analyze_pages` **):** This is the heart of the logical segmentation. It iterates through extracted pages, calling the intelligence functions from Section 4 to group them into cohesive LogicalDocument objects. It ensures that a 10-page "Mixed PDF" is correctly split into its constituent business components.

4. **UI Visualization Layer (** `load_pdf_into_viewer & flip_page` **):** Converts heavy PDF objects into a crisp image-based state for the Gradio UI. By using a high-density Matrix (3x3), it ensures that fine print on legal documents remains legible for the human reviewer.

In [ ]:
# ------- SECTION 5. ADVANCED PDF PROCESSING PIPELINE -------

# --- 1. CORE SEGMENTATION LOGIC ---
def analyze_pages(pages_info):  # Shared Analysis Logic
    """
    Groups individual pages into logical business units.
    Flow: Page Ingestion -> Boundary Detection -> Logical Document Creation.
    """

    logical_docs = []
    current_pages = []
    doc_counter = 0

    for i, page in enumerate(pages_info):
        if i == 0:
            # Initialize the first document type
            doc_type = classify_document_type(page.text)
            current_pages = [page]
        else:
            # Check if current page is a continuation of the previous one
            if detect_document_boundary(pages_info[i-1].text, page.text, doc_type):
                current_pages.append(page)
            else:
              # Boundary detected: Finalize the current logical document
                logical_docs.append(
                    LogicalDocument(
                        doc_id=f"doc_{doc_counter}",
                        doc_type=doc_type,
                        page_start=current_pages[0].page_num,
                        page_end=current_pages[-1].page_num,
                        text="\n\n".join(p.text for p in current_pages),
                    )
                )
                doc_counter += 1
                doc_type = classify_document_type(page.text)
                current_pages = [page]

    # Handle the final trailing document in the sequence
    if current_pages:
        logical_docs.append(
            LogicalDocument(
                doc_id=f"doc_{doc_counter}",
                doc_type=doc_type,
                page_start=current_pages[0].page_num,
                page_end=current_pages[-1].page_num,
                text="\n\n".join(p.text for p in current_pages),
            )
        )

    return pages_info, logical_docs


# --- 2. MULTI-MODAL INGESTION ROUTERS ---
def extract_and_analyze_file(file): # Aware Router
    """
    AWARE ROUTER: Detects file type and selects the appropriate ingestion path.
    """

    ext = os.path.splitext(file.name)[1].lower()

    if ext == ".pdf":
        return extract_and_analyze_pdf(file)
    elif ext in [".png", ".jpg", ".jpeg"]:
        return extract_and_analyze_image(file)
    else:
        raise ValueError(f"Unsupported file type: {ext}")


def extract_and_analyze_pdf(pdf_file) -> Tuple[List[PageInfo], List[LogicalDocument]]:
    """
    HYBRID OCR PIPELINE: Extracts digital text or triggers OCR for scanned pages.
    """

    # Capture the actual name from the Gradio file object
    original_filename = os.path.basename(pdf_file.name)


    print("📖 Starting PDF extraction and analysis for: {original_filename}")

    doc = fitz.open(pdf_file.name) # open file

    pages_info = []
    for i, page in enumerate(doc):
        text = page.get_text().strip()

        # Hybrid OCR: If no text found, render page to image and use Tesseract
        if not text:
            pix = page.get_pixmap()
            img = Image.open(io.BytesIO(pix.tobytes("png")))
            text = pytesseract.image_to_string(img)

        pages_info.append(PageInfo(page_num=i, text=text))

    doc.close()
    return analyze_pages(pages_info)


def extract_and_analyze_image(image_file): # Image Ingestion
    """Processes standalone image files via OCR."""

    print("🖼️ Processing Image:", image_file.name)

    img = Image.open(image_file.name)
    text = pytesseract.image_to_string(img)

    pages_info = [PageInfo(page_num=0, text=text)]
    return analyze_pages(pages_info)



 # --- 3. UI RENDERING LOGIC ---

# For Document Viewer in UI (Convert Uploaded PDF file into an image to be viewed in Gradio UI)
def load_pdf_into_viewer(selected_file):
    """
    Loads all pages into state, but only displays the first one.
    Renders PDF pages to crisp images for the Gradio interface.
    """

    if not selected_file or not os.path.exists(str(selected_file)):
        return None, {"current_page": 0, "images": []}, "**Page 0 of 0**"

    try:
        doc = fitz.open(selected_file)
        images = []
        # Matrix(3, 3) + csRGB ensures crisp black text (fixes faded look)
        for page in doc:
            pix = page.get_pixmap(matrix=fitz.Matrix(3, 3), colorspace=fitz.csRGB)
            img = Image.open(io.BytesIO(pix.tobytes("png")))
            images.append(img)
        doc.close()

        if not images:
            return None, {"current_page": 0, "images": []}, "**Empty PDF**"

        return images[0], {"current_page": 0, "images": images}, f"<center>**Page 1 of {len(images)}**</center>"
    except Exception as e:
        print(f"Viewer Error: {e}")
        return None, {"current_page": 0, "images": []}, "**Error loading**"


def flip_page(direction, state):
    """Navigates through the images stored in state."""
    images = state.get("images", [])
    current = state.get("current_page", 0)

    if not images:
        return None, state, "**Page 0 of 0**"

    if direction == "next":
        current = min(current + 1, len(images) - 1)
    else:
        current = max(current - 1, 0)

    state["current_page"] = current
    indicator = f"**Page {current + 1} of {len(images)}**"
    return images[current], state, f"<center>**Page {current + 1} of {len(images)}**</center>"


print("✅ SECTION 5. ADVANCED PDF PROCESSING PIPELINE COMPLETE.")


✅ SECTION 5. ADVANCED PDF PROCESSING PIPELINE COMPLETE.


# **SECTION 6. INTELLIGENT CHUNKING WITH METADATA PRESERVATION**

**Logic and Flow Analysis**

This section defines the Granular Transformation Layer. After a document has been logically segmented (e.g., separating an Invoice from a Contract), the text must be broken down into "chunks" that fit the context window of an LLM while ensuring that the "provenance" (where the data came from) is never lost.

<br>

The logic flow provides two distinct pathways:
1. **Semantic Sliding Window (**`chunk_document_with_metadata`**):** A custom-built algorithm that ensures no information is lost at the boundaries of chunks by creating an "overlap." It uses a calculated stride to maintain context across segments.

2. **LlamaIndex Orchestration (**`chunk_with_llama_index` **):** An alternative high-level path using the `SentenceSplitter`. This is superior for complex documents as it respects paragraph and sentence boundaries, preventing a chunk from being cut off in the middle of a critical legal clause.

3. **Metadata Injection:** This is the "Secret Sauce" of the platform. Every chunk—no matter how small—is stamped with its `doc_type`, `doc_id`, and `page_range`. This ensures that in the retrieval phase (Section 7), the system can filter out irrelevant document types with 100% precision.

In [ ]:
# ------- SECTION 6. INTELLIGENT CHUNKING WITH METADATA PRESERVATION -------

# --- 1. CUSTOM SLIDING WINDOW CHUNKING ---
def chunk_document_with_metadata(logical_doc: LogicalDocument,
                                chunk_size: int = 500,
                                overlap: int = 100) -> List[ChunkMetadata]:
    """
    Chunk a logical document while preserving rich metadata.
    Uses sliding window with overlap for better context.

    Ensures 'overlap' to prevent loss of context at chunk boundaries.
    """
    chunks_metadata = []
    words = logical_doc.text.split()

    # Case A: Document is smaller than the threshold
    if len(words) <= chunk_size:
        # Document is small enough to be a single chunk
        chunk_meta = ChunkMetadata(
            chunk_id=f"{logical_doc.doc_id}_chunk_0",
            doc_id=logical_doc.doc_id,
            doc_type=logical_doc.doc_type,
            chunk_index=0,
            page_start=logical_doc.page_start,
            page_end=logical_doc.page_end,
            text=logical_doc.text
        )
        chunks_metadata.append(chunk_meta)

    # Case B: Multi-chunk split with sliding window
    else:
        # Create overlapping chunks
        stride = chunk_size - overlap
        for i, start_idx in enumerate(range(0, len(words), stride)):
            end_idx = min(start_idx + chunk_size, len(words))
            chunk_text = ' '.join(words[start_idx:end_idx])

            # Calculate which pages this chunk spans
            # (simplified - in production, track more precisely)
            chunk_position = start_idx / len(words)
            page_range = logical_doc.page_end - logical_doc.page_start
            relative_page = int(chunk_position * page_range)
            chunk_page_start = logical_doc.page_start + relative_page
            chunk_page_end = min(chunk_page_start + 1, logical_doc.page_end)

            chunk_meta = ChunkMetadata(
                chunk_id=f"{logical_doc.doc_id}_chunk_{i}",
                doc_id=logical_doc.doc_id,
                doc_type=logical_doc.doc_type,
                chunk_index=i,
                page_start=chunk_page_start,
                page_end=chunk_page_end,
                text=chunk_text
            )
            chunks_metadata.append(chunk_meta)

            if end_idx >= len(words):
                break

    return chunks_metadata


# --- 2. LLAMA-INDEX ADVANCED CHUNKING ---
def chunk_with_llama_index(logical_doc: LogicalDocument,
                           chunk_size: int = 500,
                           chunk_overlap: int = 100) -> List[Document]: # Chunk Metadata
    """
    Alternative: Use LlamaIndex's advanced chunking with metadata.
    """
    # Create LlamaIndex document with metadata
    doc = Document(
        text=logical_doc.text,
        metadata={
            "doc_id": logical_doc.doc_id,
            "doc_type": logical_doc.doc_type,
            "page_start": logical_doc.page_start,
            "page_end": logical_doc.page_end,
            "source": f"{logical_doc.doc_type}_document"
        }
    )

    # Use LlamaIndex's sentence splitter for better chunking
    # Sentence-aware splitter prevents cutting mid-sentence
    splitter = SentenceSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        paragraph_separator="\n\n",
        separator=" ",
    )

    # Create nodes (chunks) from document
    nodes = splitter.get_nodes_from_documents([doc])

    # Convert to our ChunkMetadata format for consistency
    chunks_metadata = []
    for i, node in enumerate(nodes):
        chunk_meta = ChunkMetadata(
            chunk_id=f"{logical_doc.doc_id}_chunk_{i}",
            doc_id=logical_doc.doc_id,
            doc_type=logical_doc.doc_type,
            chunk_index=i,
            page_start=node.metadata.get("page_start", logical_doc.page_start),
            page_end=node.metadata.get("page_end", logical_doc.page_end),
            text=node.text
        )
        chunks_metadata.append(chunk_meta)

    return chunks_metadata


# --- 3. BATCH PROCESSOR ---
def process_all_documents(logical_docs: List[LogicalDocument],
                         use_llama_index: bool = False) -> List[ChunkMetadata]:
    """
    Orchestrates (Process) all logical documents into chunks with metadata.
    Can use either custom or LlamaIndex chunking.
    """
    all_chunks = []

    for logical_doc in logical_docs:
        if use_llama_index:
            chunks = chunk_with_llama_index(logical_doc)
        else:
            chunks = chunk_document_with_metadata(logical_doc)

        logical_doc.chunks = chunks  # Store reference
        all_chunks.extend(chunks)
        print(f"📄 {logical_doc.doc_type}: Created {len(chunks)} chunks")

    return all_chunks


print("✅ SECTION 6. INTELLIGENT CHUNKING WITH METADATA PRESERVATION COMPLETE.")


✅ SECTION 6. INTELLIGENT CHUNKING WITH METADATA PRESERVATION COMPLETE.


# **SECTION 7. QUERY ROUTING AND INTELLIGENT RETRIEVAL**

 **Logic and Flow Analysis**

This section serves as the **Precision Layer** of the system. Its primary goal is to solve "Context Contamination" which is a common failure in standard RAG where a query about an "Invoice date" might accidentally pull data from a "Bank Statement" just because they share similar keywords.

<br>

The logic flow implements Siloed Vector Search:
1. **Semantic Query Routing (** `predict_query_document_type` **):** Before searching the database, the system uses the LLM as a "Traffic Controller." It analyzes the user's intent to predict which specific document category (e.g., Mortgage Contract) holds the answer. The use of Regex parsing ensures that even if the LLM is verbose, the system extracts only the valid JSON routing instruction.

2. **Segregated Indexing (** `build_indices` **):** Unlike basic RAG which creates one giant bucket of data, this system builds a primary FAISS index and several sub-indices (silos) based on document types. This allows for lightning-fast, high-precision searches within a specific document class.

3. **Adaptive Retrieval (** `retrieve` **):** This function makes a real-time decision. If the Router has high confidence (>0.7), the search is restricted to the specific "silo." If confidence is low, it falls back to a global search. This "Hybrid Approach" maximizes accuracy for specific questions while maintaining flexibility for general queries.

In [ ]:
# ------- SECTION 7. QUERY ROUTING AND INTELLIGENT RETRIEVAL -------


# --- 1. THE ROUTER (INTENT ANALYSIS) ---
def predict_query_document_type(query: str, llm=None) -> Tuple[str, float]:
    """
    Predict which document type is most likely to contain the answer.
    Uses the currently active LLM from the Orchestrator.
    Dynamically applies Mistral [INST] tags only when necessary.
    """

    # Use the passed LLM, or fall back to the global Settings.llm
    active_llm = llm or Settings.llm

    # Extract model name for logging (handles LlamaIndex LLM objects)
    model_name = getattr(active_llm,
                 "model_name",
                 "AI Engine")

    print(f"🧠 Routing via {model_name}...")

    # --- STEP 1: Detect Mistral ---
    # This prevents formatting errors when switching between Gemini and Mistral
    is_mistral = "Mistral" in model_name

    # --- LOGGING THE CHOSEN LLM ---
    log_entry = {
        "timestamp": datetime.now().strftime("%H:%M:%S"),
        "event": "ROUTING_ATTEMPT",
        "model_used": model_name,
        "query_preview": query[:30] + "..."
    }
    audit_logs.append(log_entry)
    print(f"🧠 Routing via {model_name}...")

    raw_prompt = f"""
    Analyze this query and predict which document type would most likely contain the answer.

    Query: "{query}"

    Choose the MOST LIKELY type from:
    - Resume: Career, experience, education, skills, employment history
    - Contract: Terms, agreements, obligations, parties, legal terms
    - Mortgage Contract: Home loan, property financing, mortgage terms, interest rates
    - Invoice: Payments, amounts due, billing, charges, invoiced items
    - Pay Slip: Salary, wages, deductions, earnings, pay period
    - Lender Fee Sheet: Loan fees, closing costs, origination fees, lender charges
    - Land Deed: Property ownership, deed information, property description, title
    - Bank Statement: Account balance, transactions, deposits, withdrawals
    - Tax Document: Tax information, W2, 1099, tax returns, tax amounts
    - Insurance: Coverage, policy details, premiums, claims
    - Report: Analysis, findings, conclusions, research data
    - Letter: Communications, requests, notifications, correspondence
    - Form: Applications, submitted data, form fields
    - ID Document: Personal identification, ID numbers, identity verification
    - Medical: Health information, medical conditions, prescriptions
    - Other: General or unclear

    Respond in JSON format:
    {{"type": "DocumentType", "confidence": 0.85}}

    Confidence should be between 0.0 and 1.0
    """

    # --- STEP 2: Wrap Prompt for Mistral ---
    # Gemini ignores these tags, but Mistral requires them to follow instructions
    if is_mistral:
        final_prompt = f"[INST] {raw_prompt} [/INST]"
    else:
        final_prompt = raw_prompt


    try:
        # 1. Use the wrapped promptthe generic .complete() method which works for Gemini, Phi-2, TinyLlama
        response = active_llm.complete(final_prompt).text.strip()

        # 2. Use Regex Parsing to find the JSON block.
        # This prevents the "line 1 column 1" error if the LLM adds conversational text.
        # Essential to strip 'conversational' text if the LLM adds it
        json_match = re.search(r'\{.*\}', response, re.DOTALL)

        if json_match:
            # 3. EXTRACT the string found by regex
            raw_json_str = json_match.group()

            # 4. REPAIR the string (Fixes missing quotes/commas from 4-bit Mistral)
            repaired_json = repair_json(raw_json_str)

            # 5. LOAD the cleaned JSON
            result = json.loads(repaired_json)

            doc_type = result.get("type", "Other")
            confidence = result.get("confidence", 0.5)

            print(f"✅ Router assigned: {doc_type} ({confidence*100:.1f}%)")
            return doc_type, confidence
        else:
            raise ValueError("No JSON pattern found in response text")

    except Exception as e:
        print(f"🎯 Routing fallback used. Error: {e}")
        return "Other", 0.0


# --- 2. THE INTELLIGENT RETRIEVER (VECTOR ENGINE) ---
class IntelligentRetriever:
    """
    Advanced retrieval system with metadata filtering and query routing.
    """

    def __init__(self):
        self.index = None
        self.chunks_metadata = [] # Master list of all chunks from all files
        self.doc_type_indices = {} # Map of indices per document type

    def build_indices(self, new_chunks: List[ChunkMetadata]):
        """
        Builds or UPDATES FAISS indices.
        Ensures new chunks are added to the existing database.
        """
        print(f"🔨 Processing {len(new_chunks)} new chunks for the vector index...")

        # 1. Create embeddings only for the NEW chunks
        texts = [chunk.text for chunk in new_chunks]
        embeddings_list = Settings.embed_model.get_text_embedding_batch(texts, show_progress=True)
        new_embeddings = np.array(embeddings_list).astype('float32')
        dim = new_embeddings.shape[1]

        # Store embeddings in metadata for these new chunks
        for i, chunk in enumerate(new_chunks):
            chunk.embedding = new_embeddings[i]

        # --- TIER 1: GLOBAL INDEX (APPEND MODE) ---
        if self.index is None:
            self.index = faiss.IndexFlatL2(dim)

        self.index.add(new_embeddings)

        # IMPORTANT: Append new chunks to the master metadata list
        # Prevents previous files from disappearing
        self.chunks_metadata.extend(new_chunks)

        # --- TIER 2: SEGREGATED INDICES (SILOS) ---
        # Updates the silos to include the new data
        doc_types = set(chunk.doc_type for chunk in new_chunks)

        for doc_type in doc_types:
            # Find indices of the new chunks that match this type
            # Reference the full self.chunks_metadata to rebuild the mapping correctly
            all_type_indices = [idx for idx, chunk in enumerate(self.chunks_metadata)
                                if chunk.doc_type == doc_type]

            if all_type_indices:
                # Rebuild the specific silo index for this type
                # (FAISS IndexFlatL2 is fast enough to rebuild for specific silos)
                type_embeddings = np.array([self.chunks_metadata[i].embedding for i in all_type_indices]).astype('float32')

                type_index = faiss.IndexFlatL2(dim)
                type_index.add(type_embeddings)

                self.doc_type_indices[doc_type] = {
                    'index': type_index,
                    'mapping': all_type_indices  # Maps back to the updated master list
                }

        print(f"✅ Database updated. Total Chunks: {len(self.chunks_metadata)}")



    def retrieve(self, query: str, k: int = 4,
                filter_doc_type: Optional[str] = None,
                auto_route: bool = True) -> List[Tuple[ChunkMetadata, float]]:
        """
        Retrieve relevant chunks with optional filtering and routing.
        Smart Retrieval Logic: Routes to silo if confidence > 0.7, else searches global index.
        """


        # 1. GENERATE QUERY EMBEDDING - Use Settings.embed_model.get_query_embedding
        # FAISS expects a 2D numpy array (float32)
        # Wrap the single embedding in a list
        query_vec = Settings.embed_model.get_query_embedding(query)
        query_embedding = np.array([query_vec]).astype('float32')

        # Variables to store search results
        chunk_indices = []
        distances = []


        # 2. SELECTION (ROUTING) LOGIC (Which index to search?)
        # CASE A: User manually selected a specific filter (and it's not "All")
        if filter_doc_type and filter_doc_type.lower() != "all" and filter_doc_type in self.doc_type_indices:
            print(f"🔍 Searching specific silo: {filter_doc_type}")
            type_data = self.doc_type_indices[filter_doc_type]
            D, I = type_data['index'].search(query_embedding, k)

            # Map the silo-specific index back to the master self.chunks_metadata list
            chunk_indices = [type_data['mapping'][i] for i in I[0] if i != -1]
            distances = D[0][:len(chunk_indices)]

        # CASE B: Auto-Route is enabled (AI guesses the document type)
        elif auto_route:
            predicted_type, confidence = predict_query_document_type(query)

            # If AI is confident and the silo exists, search the silo
            if confidence > 0.7 and predicted_type in self.doc_type_indices:
                print(f"🎯 Auto-routed to: {predicted_type} ({confidence:.2%})")
                type_data = self.doc_type_indices[predicted_type]
                D, I = type_data['index'].search(query_embedding, k)
                chunk_indices = [type_data['mapping'][i] for i in I[0] if i != -1]
                distances = D[0][:len(chunk_indices)]
            else:
                # Fallback to global search if AI is unsure
                print(f"🌐 Low routing confidence ({confidence:.2%}). Searching all documents...")
                D, I = self.index.search(query_embedding, k)
                chunk_indices = [i for i in I[0] if i != -1]
                distances = D[0][:len(chunk_indices)]

        # CASE C: Search Everything (Filter is "All" or no filter provided)
        else:
            print("🌐 Searching global index (all files)...")
            D, I = self.index.search(query_embedding, k)
            chunk_indices = [i for i in I[0] if i != -1]
            distances = D[0][:len(chunk_indices)]

        # 3. CONVERT RESULTS TO SCORED CHUNKS
        valid_results = []
        for idx, i in enumerate(chunk_indices):
            # i is the index in the master self.chunks_metadata list
            # D[idx] is the distance (lower is better)
            dist = distances[idx]

            # Convert distance to a similarity score (0.0 to 1.0)
            score = 1.0 / (1.0 + dist)

            # Retrieve the full metadata object (preserves page numbers!)
            chunk_obj = self.chunks_metadata[i]


            #  Wrap in a Mock Object to prevent 'AttributeError' ---
            # Create a simple class-like object that has a .node and a .score
            class Retainer:
                def __init__(self, node, score):
                    self.node = node
                    self.score = score

            # Ensures the 'node' has metadata for the generator
            class MockNode:
                def __init__(self, text, metadata):
                    self.text = text
                    self.metadata = metadata
                def get_content(self): return self.text

            mock_node = MockNode(chunk_obj.text, {
                "page_start": chunk_obj.page_start,
                "page_end": chunk_obj.page_end,
                "doc_type": chunk_obj.doc_type,
                "doc_id": chunk_obj.doc_id
            })

            valid_results.append(Retainer(mock_node, score))

        return valid_results

print("✅ SECTION 7. QUERY ROUTING AND INTELLIGENT RETRIEVAL COMPLETE.")



✅ SECTION 7. QUERY ROUTING AND INTELLIGENT RETRIEVAL COMPLETE.


# **SECTION 8. ENHANCED ANSWER GENERATION WITH SOURCE ATTRIBUTION**

**Logic and Flow Analysis**

This section represents the **Output & Validation Layer.** Its goal is to transform raw data into a trustworthy business insight while providing a mechanism to measure the system's "truthfulness."

<br>

The logic flow follows a strict **Evidence-Based Generation** cycle:
1. **Context Synthesis (**  `generate_answer_with_sources` **):** Instead of just passing text to the LLM, the system builds a "Structured Context." Every chunk is prefixed with its metadata (Doc Type and Page Numbers). This forces the LLM to provide "In-Text Citations," allowing a human reviewer to verify the answer against the source document.

2. **Strict Constraint Enforcement:** The prompt is engineered with a "Closed-Domain" instruction (Answer based ONLY on provided context). This is the primary defense against hallucinations.

3. **The RAG Triad Auditor (** `evaluate_rag_performance` **):** This implements an automated quality gate. By using a "Judge LLM," the system evaluates itself on three critical metrics:
    - **Faithfulness:** Does the answer stay true to the facts in the document?
    - **Context Relevance:** Did the retriever find the right information?
    - **Answer Relevance:** Did the response actually help the user?

    

In [ ]:
# ------- SECTION 8. ENHANCED ANSWER GENERATION WITH SOURCE ATTRIBUTION -------


# --- 1. THE GENERATOR (CONTEXT-AWARE SYNTHESIS) ---
def generate_answer_with_sources(query: str,
                                retrieved_chunks: list) -> dict:
    """
    Generate answer with detailed source attribution using the active LLM.
    """
    if not retrieved_chunks:
        return {
            'answer': "I couldn't find relevant information to answer your question.",
            'sources': [],
            'confidence': 0.0,
            'context_text': ""
        }

    # 1.1 Context Preparation
    # Prefix every chunk with its 'Physical Provenance' (Type + Page)
    context_parts = []
    sources = []

    for item in retrieved_chunks:
        # LlamaIndex returns 'NodeWithScore' objects.
        # We extract the node (text) and the score.
        node = item.node if hasattr(item, 'node') else item
        score = item.score if hasattr(item, 'score') else 0.0

        # Extract metadata safely
        meta = node.metadata if hasattr(node, 'metadata') else {}
        doc_type = meta.get('doc_type', 'Document')
        p_start = meta.get('page_start', '?')
        p_end = meta.get('page_end', '?')
        text_content = node.get_content() if hasattr(node, 'get_content') else str(node)

        header = f"[Source: {doc_type}, Pages {p_start}-{p_end}]"
        context_parts.append(f"{header}\n{text_content}\n")

        sources.append({
            'doc_type': doc_type,
            'pages': f"{p_start}-{p_end}",
            'relevance': f"{score:.2%}"
        })

    context = "\n".join(context_parts)


    # 1.2 Prompt Engineering
    # Generate answer
    prompt = f"""
    You are a helpful AI Docyment Assistant. Use the provided context to answer the question.
    Be specific and cite which document type and pages support your answer.

    Context:
    {context}

    Question: {query}

    Instructions:
    1. Answer based ONLY on the provided context
    2. Mention which document type(s) contain the information
    3. Be concise but complete
    4. If the context doesn't contain enough information, say so

    Answer:
    """

    try:
        # Use the universal LlamaIndex LLM object
        response = Settings.llm.complete(prompt)

        # Explicitly calculate avg_score using 'item' to avoid 'chunk not defined'
        total_score = 0.0
        for item in retrieved_chunks:
            total_score += getattr(item, 'score', 0.0)
        avg_score = total_score / len(retrieved_chunks)

        return {
            'answer': response.text.strip(),
            'sources': sources,
            'context_text': context,
            'confidence': avg_score,
            'chunks_used': len(retrieved_chunks)
        }


    except Exception as e:
        print(f"Answer generation error: {e}")
        return {
            'answer': f"Error generating answer: {str(e)}",
            'sources': sources,
            'confidence': 0.0,
            'context_text': context
        }


# --- 2. THE AUDITOR (PERFORMANCE METRICS) ---
def evaluate_rag_performance(query, context, answer):
    """
    The 'Judge LLM' logic: Evaluates the RAG Triad in JSON format.
    Ensures high-fidelity output and detects potential hallucinations.

    RAG Triad:
    Faithfulness, Answer Relevance, and Context Relevance.
    """
    prompt = f"""
    Act as an AI Quality Auditor. Rate this RAG response (1-5 scale).

    Query: {query}
    Context: {context}
    Answer: {answer}

    Rate the following from 1-5 (5 is best) in JSON format:
    1. Faithfulness (Is the answer supported ONLY by the context?)
    2. Context Relevance (Is the retrieved context useful for the query?)
    3. Answer Relevance (Does the answer actually address the user's question?)

    Respond ONLY in JSON: {{"faithfulness": 5, "relevance": 4, "answer_relevance": 5}}
    """
    try:
        # Use the universal LlamaIndex LLM object
        response = Settings.llm.complete(prompt).text.strip()

        # JSON extraction
        json_match = re.search(r'\{.*\}', response, re.DOTALL)

        if json_match:
            return json.loads(json_match.group())
        else:
            raise ValueError("No valid JSON found in Auditor response")

    except Exception as e:
        print(f"⚠️ Audit Evaluation Error: {e}")
        # Default fallback to 0 if the LLM fails to judge itself
        return {"faithfulness": 0, "relevance": 0, "answer_relevance": 0}

print("✅ SECTION 8. ENHANCED ANSWER GENERATION WITH SOURCE ATTRIBUTION COMPLETE.")



✅ SECTION 8. ENHANCED ANSWER GENERATION WITH SOURCE ATTRIBUTION COMPLETE.


# **SECTION 9. ENHANCED DOCUMENT STORE**

**Logic and Flow Analysis**

This section defines the **Central Intelligence Hub** of the platform. The `EnhancedDocumentStore` class acts as the "Manager" that encapsulates all previously defined logic (OCR, Classification, Chunking, and Indexing) into a single, unified object.

<br>

The logic flow follows a **State-Machine Architecture:**
1. **Unified Ingestion Pipeline (** `process_file`**):** This is the primary entry point. It triggers a "Hard Reset" on every new upload to prevent data leakage between sessions. It then orchestrates the sequential flow:

    *Extraction -> Boundary Detection -> Metadata-Rich Chunking -> Vector Index Building.*

2. **Telemetry & Dashboarding:** Upon completion, the store generates `processing_stats`. This metadata is used to populate the UI dashboard, giving the user immediate feedback on how many "Logical Documents" (e.g., Invoices vs. Contracts) the AI found within their upload.

3. **Semantic Query Routing (** `query` **):** This function serves as the bridge between the UI and the Retriever. It handles the "Auto-Route" logic, deciding whether to search the entire vector space or target a specific document silo based on the user's intent.

4. **UI Data Serialization (** `get_document_structure` **):** Provides a structured summary for the Gradio interface, mapping internal dataclasses to human-readable labels and page numbers.

In [ ]:
# ------- SECTION 9. ENHANCED DOCUMENT STORE -------
class EnhancedDocumentStore:
    """
    Manages the complete document processing and retrieval pipeline.

    Stateful manager for document processing, storage, and retrieval.
    Orchestrates the lifecycle from raw bytes to searchable intelligence.
    """

    def __init__(self):
        # State Variables
        self.pages_info = []
        self.logical_docs = []
        self.chunks_metadata = []
        self.retriever = IntelligentRetriever()

        # Metadata & Analytics
        self.is_ready = False
        self.processing_stats = {}
        self.filename = None


    # --- PRIMARY INGESTION PIPELINE ---
    def process_pdf(self, pdf_file, filename: str = "document.pdf"):
        """
        Unified processing pipeline: Orchestrates Section 4 through Section 7.
        Handles both digital PDFs and scanned Images.
        """
        self.filename = filename
        self.is_ready = False
        start_time = datetime.now()


        # Step 1: File Type Routing. Get file extension
        ext = filename.split('.')[-1].lower()

        try:
          # --- THE ROUTER LOGIC (Decision Gate) ---
          # Step 1: Append instead of Overwrite ---
          # Extract new info but add it to our existing lists
          new_pages, new_logical_docs = extract_and_analyze_pdf(pdf_file)

          self.pages_info.extend(new_pages)
          self.logical_docs.extend(new_logical_docs)

          # Step 2: Chunking
          new_chunks = process_all_documents(new_logical_docs)
          self.chunks_metadata.extend(new_chunks) # Add new chunks to the master list

          # --- Update Index - Not recreate) ---
          # Ensure build_indices function is capable of adding nodes
          # or if using VectorStoreIndex: self.vector_index.insert_nodes(new_nodes)
          self.retriever.build_indices(new_chunks)

          # Step 4: Telemetry (Update stats for total database)
          process_time = (datetime.now() - start_time).total_seconds()
          self.processing_stats = {
              'filename': filename,
              'total_pages': len(self.pages_info), # Total pages in entire system
              'documents_found': len(self.logical_docs),
              'total_chunks': len(self.chunks_metadata),
              'document_types': list(set(doc.doc_type for doc in self.logical_docs)),
              'processing_time': f"{process_time:.1f}s"
          }

          self.is_ready = True
          return True, self.processing_stats

        except Exception as e:
            return False, {'error': str(e)}



    # --- 1. THE INGESTION ENGINE ---
    def process_file(self, file):
        """
        Executes the full pipeline: Extract -> Segment -> Chunk -> Index.
        Ensures a hard reset to maintain data privacy between uploads.
        """

        self.filename = os.path.basename(file.name)
        start = time.time()

        print(f"⚙️ Orchestrator: Starting pipeline for {self.filename}")

        # Step 1: Extract NEW content
        new_pages, new_docs = extract_and_analyze_file(file)

        # Step 2: Chunk and Append
        new_chunks = process_all_documents(new_docs)

        self.pages_info.extend(new_pages)
        self.logical_docs.extend(new_docs)
        self.chunks_metadata.extend(new_chunks)


        # Step 3: Index (Append Mode)
        self.retriever.build_indices(new_chunks)

        # Step 4: Stats
        self.processing_stats = {
            "filename": self.filename,
            "total_pages": len(self.pages_info),
            "total_chunks": len(self.chunks_metadata),
            "document_types": list(set(doc.doc_type for doc in self.logical_docs)),
            "processing_time": f"{time.time() - start:.2f}s",
        }

        self.is_ready = True
        return True, self.processing_stats

    # --- 2. THE QUERY ENGINE ---
    def query(self, question: str, filter_type: Optional[str] = None,
             auto_route: bool = True, k: int = 4) -> Dict:
        """
        Query the document store with automatic global fallback.
        """
        if not self.is_ready:
            return {
                'answer': "Please upload and process a PDF first.",
                'sources': [],
                'confidence': 0.0
            }

        # Sanitize the filter: Convert "All" strings to None for global search
        search_filter = None
        if filter_type and str(filter_type).strip().lower() != "all":
            search_filter = filter_type

        # FIRST ATTEMPT: Targeted Retrieval. Retrieve relevant chunks (Section 7 - Segregated Retrieval)
        retrieved = self.retriever.retrieve(
            question, k=k,
            filter_doc_type=search_filter,
            auto_route=auto_route
        )

        # FALLBACK: If 0 results found and we used a filter, try searching EVERYTHING
        if not retrieved and search_filter is not None:
            print(f"⚠️ No results found for silo '{search_filter}'. Falling back to Global Search...")
            retrieved = self.retriever.retrieve(
                question, k=k,
                filter_doc_type=None, # Remove the filter
                auto_route=False      # Disable routing for the fallback
            )
            filter_type = "All (Fallback)"

        # GENERATE RESPONSE - (Section 8 - Evidence-based Response)
        # This function should take the list of nodes and return a dict with 'answer' and 'context_text'
        result = generate_answer_with_sources(question, retrieved)

        # Ensure 'retrieved_chunks' is in the dictionary so chat_with_status can see it
        result['retrieved_chunks'] = retrieved
        result['filter_used'] = filter_type or ('auto' if auto_route else 'none')

        # Calculate a simple confidence score for the logs
        result['confidence'] = sum([n.score for n in retrieved]) / len(retrieved) if retrieved else 0.0

        return result


    # --- 3. UI HELPER METHODS ---
    def get_document_structure(self) -> List[Dict]:
        """
        Get the document structure for UI display.
        """
        if not self.logical_docs:
            return []

        structure = []
        for doc in self.logical_docs:
            structure.append({
                'id': doc.doc_id,
                'type': doc.doc_type,
                'pages': f"{doc.page_start + 1}-{doc.page_end + 1}",  # 1-indexed for UI
                'chunks': len(doc.chunks) if doc.chunks else 0,
                'preview': doc.text[:200] + "..." if len(doc.text) > 200 else doc.text
            })

        return structure


# --- UI FILTERING LOGIC ---

def get_filtered_structure(selected_filters):
    """
    selected_filters: List of strings from the Multiselect (e.g., ["Type: Report", "File: my_doc.pdf"])
    """
    # 1. Get all logical documents from your store
    # (Using the LogicalDocument dataclass from your Section 3)
    all_docs = doc_store.logical_documents

    if not selected_filters or "All" in selected_filters:
        filtered = all_docs
    else:
        # Extract the actual values from the labels
        type_filters = [f.replace("Type: ", "") for f in selected_filters if f.startswith("Type: ")]
        file_filters = [f.replace("File: ", "") for f in selected_filters if f.startswith("File: ")]

        filtered = [
            d for d in all_docs
            if d.doc_type in type_filters or os.path.basename(d.source) in file_filters
        ]

    # 2. Build the display string
    structure_lines = ["🧬 FILTERED DOCUMENT STRUCTURE:"]
    current_file = ""
    for doc in filtered:
        fname = os.path.basename(doc.source)
        if fname != current_file:
            structure_lines.append(f"\n📂 FILE: {fname}")
            current_file = fname
        structure_lines.append(f"   └─ 🏷️ {doc.doc_type.upper()} | 📑 Pgs: {doc.page_start + 1}-{doc.page_end + 1}")

    return "\n".join(structure_lines)

print("✅ SECTION 9. ENHANCED DOCUMENT STORE COMPLETE.")


✅ SECTION 9. ENHANCED DOCUMENT STORE COMPLETE.


# **SECTION 10. BACKEND CHAT  & AUDIT  LOGIC**

**Logic and Flow Analysis**

This section acts as the **"Operational Nerve Center"** of the platform. It bridges the gap between the core AI logic and the Gradio user interface. The logic focuses on three key pillars:
**Ground-Truth Validation, Performance Analytics, and UI State Management.**

<br>

The logic flow consists of:
1. **Golden Dataset Evaluation:** Defines "Ground Truth" question-answer pairs for specific domains (Legal, Healthcare, Real Estate). This allows the system to be benchmarked against human-verified answers rather than just "guessing" if a response is good.

2. **Performance Auditing (** `run_performance_audit` **):** This is the "Diagnostic Engine." It doesn't just display answers; it measures **LLM Speed** (tokens/sec), **Latency**, and **Context Precision**. It visualizes these metrics using Seaborn/Matplotlib to help users identify bottlenecks (e.g., is the delay in the Retriever or the LLM?).

3. **Advanced Export Engine:** Implements the logic for generating professional-grade PDF reports for both Chat History and Performance Audits using `ReportLab`. This ensures that the AI's findings are "portable" for business stakeholders.

4. **Batch Processing Handler (** `process_pdf_handler` **):** Manages the "Multi-File" experience. It ensures that when a user drops 5 files into the UI, they are processed in parallel, categorized into "Smart Filters," and mapped to the PDF viewer without losing track of which text belongs to which file.

In [ ]:
# ------- SECTION 10. BACKEND CHAT  & AUDIT  LOGIC -------


# 1. GLOBAL STORE INSTANCE (Initialize The Engine)
doc_store = EnhancedDocumentStore()


# 2. - GOLDEN DATASETS (Define Ground-Truth) -
# GOLDEN DATASETS to test RAG Pipleine responses with source of truth
GOLDEN_DATASETS = {
    "Healthcare": [
        {"question": "What is the primary diagnosis?", "golden_answer": "Diagnosis of Type 2 Diabetes with neuropathy."},
        {"question": "What are the latest lab results for Glucose?", "golden_answer": "Fasting glucose was 145 mg/dL."
}
    ],
    "Legal": [
        {"question": "What is the termination notice period?", "golden_answer": "The agreement requires a 30-day written notice for termination."},
        {"question": "Who are the parties involved?", "golden_answer": "Between Acme Corp and John Smith."}
    ],
    "Real Estate": [
        {"question": "What is the total cash to close?", "golden_answer": "The estimated cash to close is $95,802."},
        {"question": "What is the loan amount and the interest rate?", "golden_answer": "The loan amount is $380,000 and the interest rate is 4.25%."},
        {"question": "Who are the applicants and what is the property address?", "golden_answer": "The applicants are John Q. Smith and Mary A. Smith. The property is 1254 Main Street, San Diego, CA 92110."
}
    ]
}

# 3. - AUDIT EVALUATOR FOR MISTRAL LLM -
def evaluate_response_audit(query: str, response: str) -> Dict:
    """
    Evaluates response quality using the current LLM.
    Uses is_mistral logic to prevent JSON parsing errors.
    """
    active_llm = Settings.llm
    model_name = getattr(active_llm, "model_name", "AI Engine")
    is_mistral = "Mistral" in model_name

    raw_audit_prompt = f"""
    Evaluate this Q&A pair:
    Query: {query}
    AI Response: {response}

    Return ONLY JSON:
    {{"score": 0.9, "reasoning": "1-sentence explanation"}}
    """

    # Apply Mistral tags if needed
    final_audit_prompt = f"[INST] {raw_audit_prompt} [/INST]" if is_mistral else raw_audit_prompt

    try:
        raw_output = active_llm.complete(final_audit_prompt).text.strip()

        # Robust JSON search
        json_match = re.search(r'\{.*\}', raw_output, re.DOTALL)

        if json_match:
            repaired = repair_json(json_match.group())
            result = json.loads(repaired)
            return result
        else:
            raise ValueError("Auditor output was not structured JSON")

    except Exception as e:
        print(f"⚠️ Audit Error: {e}")
        return {"score": 0.0, "reasoning": "Audit engine parsing failed."}


 #-- 4. PERFORMANCE AUDIT & VISUALIZATION LOGIC ---
 # Generates The Accuracy Metrics & Plots
def run_performance_audit(doc_filter, audit_num_chunks):
    """
    Calculates Speed, Chunk Metrics, and RAG Triad scores for the Audit Dashboard.
    """
    if not audit_logs:
        return "**Avg Latency:** N/A", {}, "N/A", None, [["No Data", "-", "-", "-"]]

    # Convert logs to DataFrame for filtering. Filter logs based on active UI selection
    full_df = pd.DataFrame(audit_logs)

    # 1. DEFINE SECTOR MAPPINGS
    # This ties the UI selection to the AI's classification types
    sector_map = {
        "Real Estate": [
            "Mortgage Contract", "Land Deed", "Lender Fee Sheet",
            "Pay Slip", "Tax Document", "W2", "Tax Return" # Added financial types
        ],
        "Healthcare": [
            "Medical", "Medical Report", "Insurance", "Health Form"
        ],
        "Legal": [
            "Contract", "Land Deed", "Legal Letter", "Form"
        ]
    }

    # 2. APPLY FILTERING LOGIC
    if doc_filter == "All":
        filtered_df = full_df
    elif doc_filter in sector_map:
        # Filter logs where the 'Filter_Used' matches any type in the sector list
        relevant_types = sector_map[doc_filter]
        filtered_df = full_df[full_df['Filter_Used'].isin(relevant_types)]
    else:
        # Fallback for direct document type filtering (e.g., if user selects 'Invoice' directly)
        filtered_df = full_df[full_df['Filter_Used'] == doc_filter]

    # Check if we have data after filtering
    if filtered_df.empty:
        return (
            f"**No audit data found for sector: {doc_filter}**",
            {}, "0%", None, [["No Data", "-", "-", "-"]]
        )

    # 3. CALCULATE METRICS (Using your existing logic)
    avg_latency = filtered_df['latency'].mean() if 'latency' in filtered_df else 0.0

    # Calculate Success Rate (where score > 0.7)
    if 'audit_score' in filtered_df:
        success_count = (filtered_df['audit_score'] > 0.7).sum()
        success_rate = (success_count / len(filtered_df)) * 100
    else:
        success_rate = 0.0





    # Speed: Tokens per Second (Simulated based on generation time)
    avg_tokens = 150 # Placeholder for avg response length
    tokens_per_sec = avg_tokens / avg_latency if avg_latency > 0 else 0


    # METRIC 2: Context Precision (How relevant was the retrieved data?)
    # Chunk Density: Percentage of 'audit_num_chunks' that were highly relevant
    # We use the 'Relevance' score (0-5) to determine chunk quality
    avg_relevance = filtered_df['Relevance'].mean()
    context_density = (avg_relevance / 5) * 100

    # VISUALIZATION: Efficiency Bottleneck Chart (with Speed Metric)
    plt.figure(figsize=(6, 4))
    sns.barplot(x=['Retriever', 'LLM Speed'],
                y=[avg_latency * 0.3, tokens_per_sec / 10], # Normalized for scale
                hue=['Retriever', 'LLM Speed'],
                palette="viridis",
                legend=False)
    plt.title(f"Efficiency Metrics (Chunks: {audit_num_chunks})")
    plt.savefig("bottlenecks.png")
    plt.close()

    # DATA FOR UI COMPONENT (COMPARISON TABLE)
    audit_table_data = [
        ["Generation Speed", f"{tokens_per_sec:.1f} t/s", "12.5 t/s", "Industrial"],
        ["Context Precision", f"{context_density:.0f}%", "85%", "Target"],
        ["Avg Latency", f"{avg_latency:.1f}s", "3.0s", "Target"],
        ["Chunk Retrieval", f"{audit_num_chunks}", "N/A", "Config"]
    ]

    return (
        f"**Avg Latency:** {avg_latency:.2f}s | **Speed:** {tokens_per_sec:.1f} tokens/sec",
        {"Faithfulness": avg_relevance/5, "Context Density": context_density/100},
        f"{success_rate:.1f}%",
        f"{context_density:.0f}%",
        "bottlenecks.png",
        audit_table_data
    )


# --- 5. BATCH UPLOAD & UI STATE HANDLER  TO UPLOAD & PROCESS PDF
def process_pdf_handler(file_list):
    """
    Orchestrates the ingestion of multiple files and updates UI components.
    Returns: (Status Message, Structure JSON, Structure Display, Filter Update, View Selector)
    """

    try:
      if not file_list:

              # Return empty defaults for all 6 outputs
              return (
                "⚠️ No files uploaded.",
                "[]",
                "",
                gr.update(choices=["All"], value=["All"]),
                gr.update(choices=[], value=None),
                "No file uploaded"
              )

      file_reports = []
      all_doc_types = set()
      all_filenames = []
      view_selector_choices = []

      total_pages = 0
      total_chunks = 0
      start_batch_time = datetime.now()

      for file in file_list:
          # full_path is the /tmp/gradio/... path needed for the PDF viewer
          full_path = file.name
          # fname is the clean name for the UI
          fname = os.path.basename(full_path)

          # 2. PROCESS FILE - Call the Orchestrator from Section 9
          # Pass full_path to the engine so it can actually read the bits
          success, stats = doc_store.process_pdf(file, filename=fname)

          if success:
              all_doc_types.update(stats.get('document_types', []))
              all_filenames.append(fname)

              # Create the (Label, Value) tuple for the dropdown
              view_selector_choices.append((fname, full_path))

              total_pages += stats.get('total_pages', 0)
              total_chunks += stats.get('total_chunks', 0)

              # Build a clean plain-text report for this specific file
              report = (
                f"💾  {fname}\n"
                f"   └─ 📄 Pages: {stats['total_pages']} | 🧩 Chunks: {stats['total_chunks']}\n"
                f"   └─ 🏷️ Types: {', '.join(stats['document_types'])}\n"
                f"   └─ ⏱️ Time: {stats['processing_time']}"
              )

              # Build individual file report line
              file_reports.append(report)

          else:
            file_reports.append(f"❌ {fname} | FAILED: {stats.get('error', 'Unknown Error')}")


      # --- DATA AGGREGATION for STRUCTURE VIEW LOGIC---
      # 3. STRUCTURE DATA AGGREGATION - Generate Structure Visuals
      structure_json = doc_store.get_document_structure()
      structure_lines = ["🧬 GLOBAL DOCUMENT STRUCTURE:"]
      current_file = ""

      for doc in structure_json:
          doc_source = doc.get('source') or doc.get('filename') or doc.get('file_name') or "Unknown File"

          if doc_source != current_file:
              # Clean up path if it's a full /tmp/ path
            display_name = os.path.basename(doc_source)
            structure_lines.append(f"\n📂 FILE: {display_name}")
            current_file = doc_source

          structure_lines.append(f"   └─ 🏷️ {doc['type'].upper()} | 📑 Pgs: {doc['pages']} | 🧩 {doc['chunks']} chunks")

      structure_display = "\n".join(structure_lines)

      # CONSTRUCT THE MAIN STATUS LOG
      batch_time = (datetime.now() - start_batch_time).total_seconds()
      joined_reports = "\n\n".join(file_reports)

      status_msg = f"""
  ================================================================
  📂 BATCH PROCESSING COMPLETE ({batch_time:.1f}s)

  {joined_reports}

  -----------------------------------------------------------------
  📊 TOTAL BATCH STATS:
  Files: {len(all_filenames)} | Pages: {total_pages} | Chunks: {total_chunks}
  ================================================================="""


      # JSON String for the Code box
      # Convert the list (structure_json) to a JSON string
      # indent=2 makes it look like a pretty-printed JSON object in the UI
      structure_json_string = json.dumps(structure_json, indent=2)

      # 4. PREPARE SMART FILTERS (Types + Files)
      # Create labels that distinguish between Document Types and Specific Files
      unique_types = sorted(list(all_doc_types))
      type_options = [f"Type: {t}" for t in unique_types]
      file_options = [f"File: {f}" for f in sorted(all_filenames)]

      # Dynamic UI Filter logic. Combine them into one list for the multiselect dropdown
      smart_filter_choices = ["All"] + type_options + file_options


      # Update the the Search Document Filter Dropdown
      doc_type_filter_update = gr.update(choices=smart_filter_choices, value=["All"])


      # Update the "Select File to View" Dropdown
      # choices = paths, value = the first path in the list
      view_selector_update = gr.update(
          choices=view_selector_choices,
          value=view_selector_choices[0][1] if view_selector_choices else None
      )

      # 5. WIRING THE RETURN
      # Ensure outputs match the click event:
      # (status, json_code, textbox_display, doc_filter, view_selector, status_bar)
      return (
          "\n\n".join(file_reports),                         # 1. status_msg (Textbox)
          json.dumps(structure_json, indent=2),               # 2. structure_json (Code)
          "\n".join(structure_lines),                         # 3. structure_display (Textbox)
          gr.update(choices=smart_filter_choices, value=["All"]), # 4. doc_type_filter (Multiselect)
          gr.update(                                          # 5. view_selector (Dropdown)
              choices=view_selector_choices,
              value=view_selector_choices[0][1] if view_selector_choices else None
          ),
          f"✅ Successfully indexed {len(all_filenames)} files." # 6. op_status_bar (Status Label)
      )

    except Exception as e:
        print(f"Process Error: {e}")
        return f"Error: {str(e)}", "[]", "❌ Failed", gr.update(), gr.update(), "Error"


# ------- 6. EXPORT LOGIC (ReportLab) - PERFORMANCE AUDIT REPORT EXPORT (Logic for File Generation) ------- #
def handle_audit_export(audit_data):
    """
    Logic to convert your dataframe/audit results into a PDF.
    This is similar to your chat export but for the audit tab.
    """

    # Ensure audit_data is a DataFrame and not empty
    if audit_data is None or (isinstance(audit_data, pd.DataFrame) and audit_data.empty):
        return gr.update(visible=False, value=None), "⚠️ No audit data available to export."

    # Create a temporary file
    fd, path = tempfile.mkstemp(suffix=".pdf")
    os.close(fd) # Close immediately to allow ReportLab to write to it

    try:
        doc = SimpleDocTemplate(path, pagesize=letter)
        styles = getSampleStyleSheet()
        elements = []

        # 2. Add Header
        title_style = ParagraphStyle(
            'Title',
            parent=styles['Heading1'],
            fontSize=16,
            spaceAfter=20,
            alignment=1  # Center alignment
        )
        elements.append(Paragraph("AI-Powered Document Intelligence - Performance Audit Report", title_style))

        # Date and Time
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M')
        elements.append(Paragraph(f"Generated on: {current_time}", styles['Normal']))
        elements.append(Spacer(1, 20))

        # 3. Process Table Data
        # Convert DataFrame to list of lists (Header + Rows)
        # Ensure all values are strings for ReportLab
        data = [audit_data.columns.to_list()] + audit_data.values.tolist()

        # Create the Table object
        audit_table = Table(data, hAlign='CENTER')

        # Apply Industry-Standard Styling
        audit_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.darkslategray), # Header Background
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),     # Header Text
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, -1), 10),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
            ('BACKGROUND', (0, 1), (-1, -1), colors.whitesmoke),   # Body Background
            ('GRID', (0, 0), (-1, -1), 1, colors.black),           # Table Grid
            ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.lightgrey]) # Striped rows
        ]))

        elements.append(audit_table)
        elements.append(Spacer(1, 30))
        elements.append(Paragraph("<b>End of Performance Audit Report</b>", styles['Italic']))

        # 4. Build PDF
        doc.build(elements)

        # 5. Final check and return
        if os.path.exists(path):
            # We return two things: the file update and the status message
            return gr.update(value=path, visible=True, label="📥 Download Performance Audit Report"), "✅ Audit report generated successfully!"
        else:
            return gr.update(visible=False), "❌ Error: PDF file was not created."

    except Exception as e:
        print(f"Process Error: {e}")
        # Return 6 items to match the expected Gradio outputs
        return (
            f"❌ Error: {str(e)}", # 1. Status Message
            "[]",                  # 2. JSON Code
            "❌ Processing Failed", # 3. Display Text
            gr.update(),           # 4. Filter Update
            gr.update(),           # 5. View Selector
            "⚠️ System Error"      # 6. Status Bar
        )



# --- 7. REPORT GENERATION UTILITIES ---

# Wrapper to combine to generate PDF, export, and download Performance Audit. Works with 'handle_audit_export' function
def handle_audit_export_ui(audit_data):
    """
    UI Wrapper: Connects the Audit Dashboard state to the PDF downloader.
    Categorized as: UI-Backend Bridge.
    """

    # 1. Call your existing PDF generator
    # handle_audit_export usually returns (gr.update(value=path), status_msg)
    file_update, status_msg = handle_audit_export(audit_data)

    # 2. Extract the actual string path from the dictionary
    file_path = file_update.get("value") if isinstance(file_update, dict) else file_update

    if file_path and os.path.exists(file_path):
        # We return the STRING path for the DownloadButton
        # and the status message for the status bar
        return file_path, status_msg

    return None, "❌ Export failed: No data found."


# CHAT HISTORY EXPORT (Logic for PDF File Generation) & DOWNLOAD CHAT HISTORY
def export_chat_history_to_pdf(history):
    """
    Transforms the live chat session into a formatted PDF document.
    Categorized as: Data Serialization logic.
    """

    if not history or len(history) == 0:
        return None  # No file to download

    try:
        # 1. Create temporary file path
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        file_path = f"/content/AI-Powered_Document_Intelligence_Platform_Chat_History_{timestamp}.pdf"

        # 2. Setup ReportLab PDF
        doc = SimpleDocTemplate(file_path, pagesize=letter)
        styles = getSampleStyleSheet()
        elements = []

        # --- NEW: CUSTOM STYLES ---
        # Defining a specific style for the Title
        title_style = ParagraphStyle(
            'MainTitle',
            parent=styles['Heading1'],
            fontSize=20,
            textColor=HexColor("#1A5276"), # Professional Navy Blue
            alignment=1, # 0=Left, 1=Center, 2=Right
            spaceAfter=14
        )

        # Defining a style for the Header (Top Right Info)
        header_info_style = ParagraphStyle(
            'HeaderInfo',
            parent=styles['Normal'],
            fontSize=9,
            textColor=colors.grey,
            alignment=2 # Right Aligned
        )

        # ADDED THIS: Define body_style to fix your error
        body_style = ParagraphStyle(
            'BodyStyle',
            parent=styles['Normal'],
            fontSize=10,
            leading=14)



        # --- HEADER & TITLE TO ELEMENTS ---
        # timestamp/header info at the top right
        gen_time = datetime.now().strftime("%B %d, %Y | %H:%M")
        elements.append(Paragraph(f"Generated on: {gen_time}", header_info_style))
        elements.append(Paragraph("AI Document Intelligence Automation Platform", header_info_style))
        elements.append(Spacer(1, 15))

        # Main Title
        elements.append(Paragraph("AI-Powered Document Intelligence Automation Platform Chat History", title_style))

        # horizontal line (visual break)
        # Using a table with a bottom border to make a line in Platypus
        line_table = Table([[""]], colWidths=[450])
        line_table.setStyle(TableStyle([('LINEBELOW', (0,0), (-1,-1), 1, colors.black)]))
        elements.append(line_table)
        elements.append(Spacer(1, 20))

        # 3. BUILD CHAT CONTENT
        for entry in history:
            role = "USER" if entry['role'] == 'user' else "AI ASSISTANT"
            raw_text = str(entry['content'])

            # --- CLEANING LOGIC ---
            # A. Remove the Metadata Footer (Everything from '---' onwards)
            clean_text = raw_text.split('---')[0]

            # B. Remove specific symbols and markdown markers
            clean_text = clean_text.replace("**", "").replace("■", "").replace("👤", "").replace("🤖", "")

            # C. Clean up line breaks and leading/trailing whitespace
            clean_text = clean_text.strip()

            # D. Handle the "Sources" section (Optionally keep it but clean symbols)
            clean_text = clean_text.replace("🔍 Sources:", "\n<b>Sources:</b>")

            # 3. Add to Elements
            elements.append(Paragraph(f"<b>{role}:</b>", styles['Normal']))
            elements.append(Paragraph(clean_text, body_style))
            elements.append(Spacer(1, 5))

        doc.build(elements)
        return file_path

    except Exception as e:
        print(f"❌ Export Error: {e}")
        return None

print("✅ SECTION 10. BACKEND CHAT  & AUDIT  LOGIC COMPLETE.")


✅ SECTION 10. BACKEND CHAT  & AUDIT  LOGIC COMPLETE.


# **SECTION 11. CHATBOT LOGIC & ORCHESTRATION**

**Logic and Flow Analysis**

This section defines the **User Experience (UX) Engine** and serves as the primary controller for the platform. It is a high-concurrency environment that manages **Asynchronous Communication** and **Hardware State**.

<br>

The logic flow follows a **Six-Stage Execution Cycle::**
1. **Readiness & Initialization:** Ensures the Document Store is populated and sanitizes incoming metadata filters (e.g., converting Gradio lists to LlamaIndex-compatible strings).

2. **Agentic Routing & Streaming UX:** Uses Python `generators` (`yield`) to provide immediate feedback. By signaling that "AI is thinking" before the computation begins, it reduces "perceived latency" by up to 70%.

3. **Hardware-Aware Switching (VRAM Purge):** The `switch_llm` function acts as a safety gate. Because the T4 GPU cannot hold two local models simultaneously, it performs a **Deep Purge** (deleting object references and clearing the CUDA cache) before calling the Factory functions from Section 2B.

4. **Context-Aware Retrieval:** Executes the RAG query with robust metadata handling, ensuring that even if source documents vary in format, page numbers and document types are correctly extracted for the UI.

5. **Quality Audit Gate:** Immediately triggers the "RAG Triad" evaluation (Section 8) to calculate Faithfulness and Relevance scores before the user sees the answer.

6. **Telemetry & History Management:** Logs the entire interaction into `audit_logs` and manages the session state, allowing users to choose whether to keep or clear context when switching models.


In [ ]:
# ------- SECTION 11. CHATBOT LOGIC & ORCHESTRATION -------


# Chat handler with status bar update. Define how the AI thinks and responds.
def chat_with_status(message, history, doc_type_filter, auto_route, audit_num_chunks):
        """
        Combines role definitions with real-time status updates.
        Returns: Updated history (list of dicts) and status bar text.

        Stateful Chat Handler: Manages the 'Thinking Loop' and streams status updates.
        Returns: Updated history (list of dicts) and status bar text for Gradio.
        """

        if history is None:
          history = []

        log_entry = None
        start_total = time.time()

        # 1. INITIALIZATION & READINESS
        # Ensure filter is a string (hashable) for the Vector Store
        # Gradio dropdowns often pass a list like ['Contract']. LlamaIndex filters need the string 'Contract'.
        if isinstance(doc_type_filter, list) and len(doc_type_filter) > 0:
            clean_filter = str(doc_type_filter[0])
        elif doc_type_filter:
            clean_filter = str(doc_type_filter)
        else:
            clean_filter = "All"

        active_filters = [clean_filter]
        filter_label = clean_filter


        # Check System Readiness. Check if documents exist
        if not doc_store.is_ready:
            response = "📚 Please upload and process a PDF document first."
            history.append({"role": "user", "content": f"**👤 You:** {message}"})
            history.append({"role": "assistant", "content": f"**🤖 AI Docuement Assistant:** {response}"})
            yield history, "⚠️ System Not Ready"
            return

        # 2. START PROCESSING TELEMETRY
        # Responsive UI Start. Immediate UI Feedback (Streaming Yield)
        # Immediately tell the user the AI is working so the app feels responsive.
        start_total = time.time()
        routed_type = clean_filter
        routing_confidence = 1.0


        # 3. AGENTIC ROUTING (UI FEEDBACK LAYER)
        if auto_route:
            yield history, "🎯 AI is routing your query..."
            # Pass Settings '.llm' to ensure it uses the engine selected in the UI
            routed_type, routing_confidence = predict_query_document_type(message, Settings.llm)
            clean_filter = routed_type
            active_filters = [routed_type]
            filter_label = routed_type
            print(f"✅ Router assigned category: {routed_type} ({routing_confidence:.2%})")


        # 4. STREAMING ANALYTICS (Update UI to show the 'Silo' being searched)
        # Responsive UI Start
        history.append({"role": "user", "content": f"**👤 You:** {message}"})
        history.append({"role": "assistant", "content": f"**🤖 AI Document Assistant is 🧠 Analyzing {filter_label} documents...**"})
        yield history, f"⏳ Searching (Filter: {filter_label})..."

        # DEBUG PRINT: Verify what we are asking the database
        print(f"DEBUG: Querying for '{message}' with filter '{clean_filter}'")
        print("-" * 30)
        print(f"🔍 DEBUG: Sending to Database...")
        print(f"   > Query: '{message}'")
        print(f"   > Applied Filter: '{clean_filter}'")
        print(f"   > Search Depth (k): {audit_num_chunks}")
        print("-" * 30)


        try:
            # Sanitize the filter: Force it to None if it's "All" or empty
            # Ensure search_filter is None if "All" is selected to bypass metadata silos
            search_filter = None if clean_filter.strip().lower() == "all" else clean_filter

            # 5. EXECUTE CORE RETRIEVAL & GENERATION (Section 9)
            # Limit the search depth 'k' based on the user's Audit Slider
            result = doc_store.query(
                message,
                filter_type=search_filter,
                auto_route=False, # Set to False here since we already routed above
                k=int(audit_num_chunks) if audit_num_chunks else 5
            )

            # Extract the chunks correctly for the next step
            # Note: your doc_store.query likely returns chunks in a key called 'retrieved_chunks'
            chunks_to_process = result.get('retrieved_chunks', [])


            # --- SMART FALLBACK ---
            # If a filtered search returns 0, immediately try a global search
            if len(chunks_to_process) == 0 and search_filter is not None:
                print(f"⚠️ Falling back to global search for: {message}")
                result = doc_store.query(
                    message,
                    filter_type=None, # Explicitly remove the filter
                    k=int(audit_num_chunks) if audit_num_chunks else 5
                )
                chunks_to_process = result.get('retrieved_chunks', [])
                clean_filter = "All (Fallback)"


            # 6. Generate Answer (Now 'chunks_to_process' is defined)
            generation_result = generate_answer_with_sources(message, chunks_to_process)


            answer = generation_result.get('answer', "I'm sorry, I couldn't generate an answer for that.")
            context_text = generation_result.get('context_text', "")

            # 7. POST-GENERATION AUDIT (The Quality Gate)
            # Evaluate the 'RAG Triad' immediately after generation
            scores = evaluate_rag_performance(message, context_text, answer)

            # 8. LOGGING FOR PERFORMANCE DASHBOARD
            latency = time.time() - start_total

            # Safe extraction of LLM model name for logs
            active_llm = Settings.llm

            # Check various attributes where model names might be stored
            model_name = getattr(active_llm, "model_name",
                         getattr(active_llm, "model",
                         "Gemini 2.0 Flash")) # Final fallback

            # Simple cleanup for the UI
            if "gemini" in model_name.lower() and "Flash" not in model_name:
                model_name = "Gemini 2.0 Flash"


            # Inside chat_with_status when creating log_entry:
            log_entry = {
                "Timestamp": datetime.now().strftime("%H:%M:%S"),
                "Model": model_name,
                "Query": message[:50],
                "Latency_s": round(latency, 3),
                "Routed_Category": routed_type,
                "Routing_Conf": round(routing_confidence, 2),
                "Faithfulness": scores.get("faithfulness", 0),
                "Relevance": scores.get("relevance", 0),
                "Filter_Used": clean_filter # Convert list to string
            }


            audit_logs.append(log_entry) # Ensure audit_logs = [] is defined in Section 1

            # 9. FINAL UI RESPONSE CONSTRUCTION (Robust Metadata Handling)
            retrieved_chunks = result.get('retrieved_chunks', [])
            source_entries = []

            for chunk in retrieved_chunks:
                # --- DATA EXTRACTION ---
                # Handles both LlamaIndex objects AND our custom FAISS tuples
                if hasattr(chunk, 'node'):
                    # It's a LlamaIndex-style object
                    node_text = chunk.node.get_content()
                    meta = chunk.node.metadata
                elif isinstance(chunk, (list, tuple)):
                    # A ChunkMetadata & Score tuple from FAISS
                    node_text = chunk[0].text
                    meta = {
                        "page_start": getattr(chunk[0], 'page_start', '?'),
                        "doc_type": getattr(chunk[0], 'doc_type', 'Document')
                    }
                else:
                    meta = {}

                # Try to find the page number from common keys
                page = meta.get('page_start', meta.get('page_label', meta.get('page_num', '?')))
                doc_type = meta.get('doc_type', 'Document')

                source_entries.append(f"{doc_type} (p.{page})")

            # Unique sources only to avoid clutter
            unique_sources = list(set(source_entries))
            sources_text = "\n\n🔍 **Sources:** " + ", ".join(unique_sources) if unique_sources else "\n\n⚠️ **No relevant data found.**"

            # --- UPDATE CONFIDENCE CALCULATION ---
            # Ensure we calculate confidence correctly based on the object type
            def get_score(c):
                if hasattr(c, 'score'): return c.score
                if isinstance(c, (list, tuple)) and len(c) > 1: return c[1]
                return 0.0

            raw_confidence = sum([get_score(c) for c in retrieved_chunks]) / len(retrieved_chunks) if retrieved_chunks else 0.0



            # Metadata Footer
            stats_text = f"\n\n---\n*⏱️ {latency:.2f}s | 🤖 Engine: {model_name} | ✅ Faithfulness: {scores.get('faithfulness', 0)}/5*"

            # Confidence & Filter Display
            # Use the actual filter applied by the doc_store
            applied_filter = result.get('filter_used', clean_filter)
            metadata = f"\n\n*Confidence: {raw_confidence:.1%} | Filter Used: {applied_filter}*"

            # Clean up double newlines and special characters for a cleaner UI
            clean_answer = answer.replace('■', '').strip()
            full_response = f"🤖 **AI Document Assistant:** {clean_answer}{sources_text}{stats_text}{metadata}"

            # Update the last entry (the placeholder) with the final answer
            # We target the last item in the list which is the assistant's placeholder
            history[-1] = {"role": "assistant", "content": full_response}

            # Final yield to close the loop
            yield history, "✅ Response Generated"

        except Exception as e:
            error_msg = f"**🤖 AI Document Assistant:** ⚠️ Error: {str(e)}"
            print(f"❌ Chat Error: {str(e)}")
            history[-1] = {"role": "assistant", "content": error_msg}
            yield history, "❌ Search Failed"

# --- 2. CENTRAL SWITCHING & VRAM MANAGEMENT ---
def deep_purge_gpu():
    import gc
    import torch
    gc.collect()
    torch.cuda.empty_cache()

    # Remove the global reference safely
    if 'current_llm' in globals():
        # Global not deleted
        # Set to None to allow the next load to happen cleanly.
        pass
    print("🧹 GPU Memory Purged.")

def switch_llm(model_choice):
    """
    The technical engine that loads the model.
    Orchestrates the transition between LLMs while strictly managing T4 GPU VRAM.
    """

    # Declare current_llm global at the very start
    global current_llm, current_model_name, audit_logs



    # Optimization: Prevent redundant loading of the same model
    if model_choice == current_model_name:
        return f"✅ {model_choice} is already active."

    print(f"🔄 Switching to {model_choice}...")

    # --- MEMORY MANAGEMENT (PURGE- Memory Safety) ---
    # Essential for T4 GPU reliability in Google Colab
    # Clear Memory to prevent OOM (Out of Memory) Errors
    if current_llm:
        print("🧹 Purging previous model from VRAM...")

        try:
            del current_llm
            deep_purge_gpu()
            time.sleep(1) # Allow VRAM to stabilize
        except Exception as e:
            print(f"⚠️ Purge warning: {e}")
    # -------------------------------


    # --- DYNAMIC LOADING LOGIC ---
    try:
        if model_choice == "Gemini 2.0":
            current_llm = setup_gemini_llm()
            # Use object.__setattr__ to bypass Pydantic's "No field model_name" error
            object.__setattr__(current_llm, 'model_name', "Gemini 2.0 Flash")

        elif model_choice == "Mistral 7B":
            current_llm = setup_mistral_7b_llm()
            object.__setattr__(current_llm, 'model_name', "Mistral 7B")

        elif model_choice == "Phi-2":
            current_llm = setup_phi2_llm()
            object.__setattr__(current_llm, 'model_name', "Microsoft Phi-2")

        elif model_choice == "TinyLlama":
            current_llm = setup_tinyllama_llm()
            object.__setattr__(current_llm, 'model_name', "TinyLlama 1.1B")

        # 4. Re-Sync Global Settings
        current_model_name = model_choice
        Settings.llm = current_llm

        # 5. Logging & Audit
        display_name = getattr(current_llm, 'model_name', model_choice)
        audit_logs.append({
            "Timestamp": datetime.now().strftime("%H:%M:%S"),
            "Event": "LLM_SWITCH",
            "Details": f"Active Engine: {display_name}"
        })

        return f"🚀 Active Engine: {model_choice}"

    except Exception as e:
        print(f"❌ Error loading {model_choice}: {e}")
        return f"❌ Failed to load {model_choice}"


def handle_model_transition(new_choice, current_history, should_clear):
    """
    The UI manager that talks to Gradio. Backend connector for the UI dropdown.
    Switches the LLM and manages the Chatbot state.
    """
    # Switch the actual model in the backend
    switch_status = switch_llm(new_choice)

    # Decide what to do with the UI state
    if should_clear:
        # Clear the history list and show an alert
        return [], f"✅ {switch_status} | System Reset. History Cleared"

    # Keep the history and show a status update
    return current_history, f"✅ {switch_status} | Context Retained. History Kept."

print("✅ SECTION 11. CHATBOT LOGIC & ORCHESTRATION Complete")



✅ SECTION 11. CHATBOT LOGIC & ORCHESTRATION Complete


# **SECTION 12. GRADIO INTERFACE, CHAT HANDLERS, & WIRING LOGIC**

**Logic and Flow Analysis**

Section 12 is the **Nervous System** of the application. While previous sections defined how the AI "thinks" (Logic) and "remembers" (Vector Store), this section defines how the AI **"interacts"**. It uses the Gradio library to build a professional-grade web interface entirely in Python.

<br>

The core of this section is **Component Wiring**. In a complex RAG application, a single button click (like "Process Document") must trigger a chain reaction:

1. **Frontend:** The button goes into a "loading" state.

2. **Middle-ware:** The `process_pdf_handler` is called.

3. **State Management:** The `doc_store` is updated, the `viewer_state` is populated with images, and the `doc_type_filter` dropdown is refreshed with new document silos.

4. **Backend:** Metadata is extracted and formatted into JSON for the developer view.

5. **Return:** The UI updates 6+ different components simultaneously.

In [ ]:
# ------- SECTION 12. GRADIO INTERFACE, CHAT HANDLERS, & WIRING LOGIC -------


# --- JavaScript is for behavior (Auto-Scroll) ---
scroll_script = """
function() {
    const targetNode = document.querySelector('#chatbot-box');
    if (!targetNode) {
        console.log("Chatbot box not found yet...");
        return;
    }

    const observer = new MutationObserver(() => {
        // In newer Gradio, the scrollable area is usually a 'div' inside the chatbot
        const scrollContainer = targetNode.querySelector('.scrollable-auto') || targetNode.querySelector('.wrapper') || targetNode;
        if (scrollContainer) {
            scrollContainer.scrollTo({
                top: scrollContainer.scrollHeight,
                behavior: 'smooth'
            });
        }
    });

    observer.observe(targetNode, { childList: true, subtree: true });
}
"""


# ----------------------------------------------- UI LAYOUT  ------------------------------------------------------------------------------------- #

# CSS: We target only the 'header-container' for centering
# CSS: Targets only the tab navigation bar to make it look like a black menu
# Targets Download & Export buttons for Chat History & Performance Audit Report
custom_css = """
    /* Center the header text */
    .welcome-text-header-container {
        text-align: center;
        margin-bottom: 20px;
    }

    /* 1. FORCE ALL BUTTONS TO BLACK */
    /* This targets primary buttons, secondary buttons, and specific IDs */
    button.primary, button.secondary, #dark-btn, #chat-export-btn, #ingest_btn, #send_btn {
        background-color: black !important;
        background: black !important;
        color: white !important;
        border: 1px solid #444 !important;
        box-shadow: none !important;
    }

    /* Button Hover Effect */
    button.primary:hover, button.secondary:hover {
        background-color: #222 !important;
        border-color: #00d1b2 !important;
    }

    /* 2. NAVIGATION BAR (TABS) STYLING */
    /* The horizontal strip background */
    .tabs > .tab-nav {
        background-color: black !important;
        border-bottom: 2px solid #333 !important;
        padding: 8px 10px 0px 10px !important;
        display: flex !important;
        gap: 5px !important;
        border-radius: 8px 8px 0 0 !important;
    }

    /* Individual Tab Labels (Inactive) */
    .tabs > .tab-nav > button {
        background-color: #111 !important; /* Very dark grey for inactive */
        color: white !important;           /* White font */
        border: none !important;
        border-radius: 5px 5px 0 0 !important;
        padding: 10px 25px !important;
        font-weight: bold !important;
    }

    /* The Active (Selected) Tab */
    .tabs > .tab-nav > button.selected {
        background-color: black !important; /* Pure black for active */
        color: #00d1b2 !important;           /* Highlight font color */
        border-bottom: 3px solid #00d1b2 !important;
    }

    /* LABELS (The specific fix you requested) */
    .gradio-container .label {
        background-color: black !important;
        color: white !important;
        padding: 4px 10px !important;
        border-radius: 5px 5px 0 0 !important;
        border: 1px solid #444 !important;
        box-shadow: none !important;
        font-weight: bold !important;
    }

    .control-frame {
    border: 1px solid #e0e0e0;
    border-radius: 12px;
    padding: 20px;
    background-color: #fcfcfc;
    box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1);
    }
    .section-divider {
        border-top: 2px solid #3b82f6;
        margin: 15px 0;
        opacity: 0.5;
    }

    .vertical-divider {
    border-right: 2px solid #e0e0e0;
    height: 90vh; /* Fills most of the vertical screen */
    margin: 0 20px;
    align-self: center;
}
    """


def create_interface():
    # Load custom CSS for the 'Obsidian' black-and-teal theme
    with gr.Blocks(title="AI-Powered Document Intelligence Automation Platform") as demo:
      # --- 1. HEADER SECTION(CENTERED) ---
      with gr.Column(elem_classes="welcome-text-header-container"):
          gr.Markdown("# 🤖 AI-Powered Document Intelligence Automation Platform")
          gr.Markdown("### Providing assistance with document search. ✨")
          gr.Markdown("📂 Upload & Process Multi-page PDF or Scanned Image and then enter search request in chatbot")
          gr.Markdown("Accepted fiile formats: .pdf, .png, .jpg, .jpeg")
          gr.HTML("<hr style='border: 1px solid #e0e0e0;'>")

      # --- 2. THREE-PILLAR NAVIGATION TABS ---
      with gr.Tabs() as tabs:
# -------- --- TAB 1: OPERATIONS (CHATTING & VIEWING) ---
          with gr.TabItem("💬 Chat Operations", id="chat_tab"):
             with gr.Row():
 # -----------     --------   # TAB 1: OPERATIONAL CORE (LEFT TOP COLUMN: UI IMAGE)
                with gr.Column(scale=1):
                    gr.Image( # AI-Powered Document Assistant logo v2.png
                        value=LOGO_PATH,
                        width=100,
                        show_label=False,
                        container=False,
                        scale=1)

                    # --- DIVIDER ---
                    gr.HTML("<hr>")

 # -----------     --------    # TAB 1 - LEFT COLUMN: LARGE LANGUAGE MODEL (LLM) SELECTION
                    with gr.Row():
                      gr.Markdown("# 🧠 Large Language Models (LLM)")

                    with gr.Row():
                      # Status indicator
                      engine_status = gr.Markdown("*Status: Ready*")

                    with gr.Row():
                      # Create the choice button
                      llm_selector = gr.Dropdown(
                          choices=["Gemini 2.0", "Mistral 7B", "Phi-2", "TinyLlama"],
                          label="Select LLM Engine",
                          value="Gemini 2.0",
                          scale=1,
                          container=False)
                      clear_on_switch_checkbox = gr.Checkbox(label="Clear History on Switch")

                    # --- DIVIDER ---
                    gr.HTML("<hr>")



                    # DOCUMENT PROCESSING CENTRAL
                    with gr.Row():
                      gr.Markdown("## 📂 Document Processing Central")

                    # FILE_UPLOAD, INGEST_BTN (PROCESS DOCUMENT), CLEAR_ALL_BTN, DOC_TYPE_FILTER, STATUS_OUTPUT
                    with gr.Row():
                      gr.Markdown("Upload file(s) and press Process Document button")

                    with gr.Row():
                      file_upload = gr.File(
                        label="Upload Multi-page PDF or Scanned Image",
                        file_count="multiple", #Enable muliple
                        file_types=[".pdf", ".png", ".jpg", ".jpeg"],
                        interactive=True,
                        type="filepath")


                    with gr.Row():
                      ingest_btn = gr.Button("🔄 Process Document", variant="primary", interactive=True, scale=1 )
                      clear_all_btn = gr.Button("🗑️ Clear All", variant="primary", interactive=True, scale=1)

                    # --- DIVIDER ---
                    gr.HTML("<hr>")


                    # PROCESSING STATUS & METADATA
                    with gr.Row():
                      # This shows the bullet points of document pages created in 'structure_display' string in def 'process_pdf_handler'.
                      status_output = gr.Textbox(
                          label="Processing Status & Metadata",
                          lines=15, # Increased height
                          elem_classes="status-window",
                          interactive=False,
                          placeholder="Technical details will appear here after upload...",
                          visible=True,
                          elem_id="status-box")   # ID for custom styling

                    # --- DIVIDER ---
                    gr.HTML("<hr>")


                    # VIEW DOCUMENT
                    with gr.Row():
                      gr.Markdown("# 📄 Document Preview")

                    with gr.Row():
                      # View the PDF pages as images in the UI
                      # Fixed height viewer prevents layout shifts
                      doc_viewer = gr.Image(
                              label="Page Viewer",
                              type="pil",
                              interactive=False,
                              height=550)

                    with gr.Row():
                      prev_btn = gr.Button("⬅️ Previous", scale=1)
                      # Indicator shows: Page 1 of 10
                      next_btn = gr.Button("Next ➡️", scale=1)

                    with gr.Row():
                      page_indicator = gr.Markdown("## <center>Page 0 of 0</center>")

                      op_status_bar = gr.Markdown("**Status:** Ready")
                      # Hidden State to store the PDF pages and current index
                      viewer_state = gr.State({"current_page": 0, "images": []})
                      filename_debug_output = gr.Textbox(label="Uploaded Filename (Debug)", visible=False, lines=1, interactive=False) # ADDED DEBUG TEXTBOX



# -----------     --------   # TAB 1 - RIGHT COLUMN: AI-POWERED DOCUMENT INTELLIGENCE CHATBOT INTERFACE

                with gr.Column(scale=2):
                    gr.Markdown("## AI-Powered Document Intelligence Chatbot")

                    # Chatbot Design
                    chatbot = gr.Chatbot(
                      label="AI Document Assistant",
                      height=1000,
                      show_label=True,
                      value=[{"role": "assistant", "content": "**🤖 AI Document Assistant:** 👋 Welcome! Upload files in the 📂 Upload & Process Documents tab to begin. 🚀"}],
                      elem_id="chatbot-box",
                      autoscroll=True,
                      render_markdown=True) # Processes the **bold** text

                    # --- DIVIDER ---
                    gr.HTML("<hr>")

                    with gr.Row():
                        msg_input = gr.Textbox(show_label=False, placeholder="Ask a question about your docs...", scale=8, container=True)

                    with gr.Row():
                        send_btn = gr.Button("🚀Send", scale=1, variant="primary", interactive=True)
                        chat_download_btn = gr.DownloadButton(
                               "📤 Download Chat History (PDF)", # Button the user clicks to start the export
                               visible=True,
                               interactive=True,
                               elem_id="chat-export-btn",
                               variant="primary",
                               scale=1)

                        # This component holds the actual file once generated
                        # Visible=False until the file is ready
                        chat_download_file = gr.File(label="Download Ready", visible=False, scale=1)

                    with gr.Row():
                        example_btn1 = gr.Button("📝 Summary", variant="primary", interactive=True, scale=1)
                        example_btn2 = gr.Button("💰 Find Amounts", variant="primary", interactive=True, scale=1)
                        clear_chat_btn = gr.Button("🗑️ Clear Chat", variant="primary", interactive=True, scale=1)

# -----------  # --- TAB 2: RAG CONFIGURATIONS & DOCUMENT(s) FILTERS ---
          with gr.TabItem("⚙️ Configurations & 📂 Filters", id="Config_filters_tab"):
             with gr.Row():


 # -----------     --------   # TAB 2 - LEFT COLUMN: DOCUMENT STRUCTURE
                with gr.Column(scale=2):
                    # DOCUMENT STRUCTURE
                    gr.Markdown("# 🧬 Processed Document Breakdown")

                    with gr.Row():
                          gr.Markdown("""
                            These view displays the breakdown of your file. Our system identifies
                            identifies distinct sub-documents types (e.g., an Invoice followed by a Lease)
                            within a single upload, mapping the specific page ranges and initial content previews
                            to ensure the retriever (search) knows exactly where each piece of information originated.
                            """)

                    # --- DIVIDER ---
                    gr.HTML("<hr>")


                    # Human-Readable Text output
                    with gr.Row():
                        gr.Markdown("### Document Structure")

                    with gr.Row():
                        gr.Markdown("Identifies distinct sub-documents and page ranges within your file.")

                    with gr.Row():
                        structure_output_textbox = gr.Textbox(label="Text Output", visible=True, scale=3, lines=8)


                    # --- DIVIDER ---
                    gr.HTML("<hr>")

                    # Developer JSON output
                    with gr.Row():
                        gr.Markdown("### Developer Document Structure")

                    with gr.Row():
                        gr.Markdown("Machine-ready schema for debugging and database integration.")

                    with gr.Row():
                         # This shows the actual raw JSON data "ADDED code" in def 'process_pdf_handler'.
                        structure_output_code = gr.Code(label="JSON Output", language="json", lines=25, interactive=False, elem_id="structure-json-box")


 # -----------     --------   # TAB 2 - RIGHT COLUMN: FILTERS  & RAG CONFIGURATIONS
                with gr.Column(scale=2):
                    gr.Image( # Document Filter and RAG.png
                        value=CONFIG_FILTER_PATH,
                        show_label=False,
                        container=False,
                        scale=1)

                    # --- DIVIDER ---
                    gr.HTML("<hr>")


                    # FILTER DOCUMENTS
                    with gr.Row():
                        gr.Markdown("# Filters")

                    with gr.Row():
                        gr.Markdown("Filter AI Document Assistant responses or document preview by File or Document Type.")

                    with gr.Row():
                        view_selector = gr.Dropdown(label="Select File to View", choices=[],scale=2)
                    with gr.Row():
                        doc_type_filter = gr.Dropdown(
                            choices=["All"],
                            label="Filter By Document (File) & Type:",
                            value="All",
                            interactive=True,
                            multiselect=True,
                            scale=2)

                    # --- DIVIDER ---
                    gr.HTML("<hr>")


                    with gr.Row():
                        # RIGHT COLUMN: RAG CONFIGURATION ROW
                        gr.Markdown("# ⚙️ RAG Configuration")

                    with gr.Row():
                        gr.Markdown("To refine AI Document Assistant response, adjust Recall Chunks.")

                    with gr.Row():
                        auto_route = gr.Checkbox(value=True, label="🎯 Auto-Route Queries")

                    with gr.Row():
                        audit_num_chunks = gr.Slider(
                                  minimum=1,
                                  maximum=10,
                                  value=4,
                                  step=1,
                                  label="📊 Recall Chunks",
                                  info="Determines how many chunks are analyzed for precision.")


# -----------    # --- TAB 3: AUDIT & GROUND TRUTH ---
          with gr.TabItem("⚖️ Performance Audit", id="audit_tab"):

              #Blank row for Spacing
              with gr.Row():


                  # LEFT COLUMN: SECTOR FILTER PERFORMANCE AUDIT
                  with gr.Column(scale=1):
                      gr.Markdown("### ⚙️ PERFORMANCE AUDIT CONFIGURATIONS")

                      with gr.Row():
                        sector_dropdown = gr.Dropdown(
                        choices=["Real Estate", "Healthcare", "Legal", "All"], # Can be optimized in future enhancements for other domains
                        label="Select Performance Audit Sector",
                        value="All",
                        interactive=True,
                        visible=True
                      )

                      # Left-Middle-Top: GRAPHS
                      with gr.Row():
                        run_audit_btn = gr.Button("🏁 Run Performance Audit", variant="primary", scale=1)
                        audit_download_btn = gr.DownloadButton("📄 Download Performance Audit Report (PDF)", variant="primary", scale=1)
                        audit_download_file = gr.File(label="📄 Download Performance Audit Report (PDF)", visible=False, interactive=False, container=True)

                  # RIGHT COLUMN: VISUALIZATION PERFORMANCE AUDIT METRICS
                  with gr.Column(scale=6):
                        gr.Markdown("# ⚖️ Performance Audit")

                        # HEADING
                        with gr.Column(scale=6):
                            gr.Markdown("### ⚙️ MONITORING & PERFORMANCE DASHBOARD")
                            gr.Markdown("### 🛠️ Industry Ground Truth Evaluation")
                            gr.Markdown("This dashboard translates raw AI-Judge scores into Permonace Audit status.")
                            gr.Markdown("--------------------------------------------------------------------------------")
                            gr.Markdown("### 📊 Live Performance")
                            gr.Markdown("--------------------------------------------------------------------------------")

                        # RIGHT COLUMN: VISUALIZATION PERFORMANCE AUDIT METRICS
                        with gr.Column(scale=6):

                          # Right-Top: LIVE PERFORMANCE
                          with gr.Row():
                            latency_stat = gr.Markdown("**Avg Latency:** -- | **Speed:** --")
                          with gr.Row():
                            audit_accuracy_gauge = gr.Label(label="RAG Triad & Context Metrics")
                            accuracy_gauge = gr.Label(label="Context Density Score")
                            bottleneck_plot = gr.Image(label="Latency vs. Token Speed")

                          # Right-Middle-Top: AUDIT TABLE
                          with gr.Row():
                            audit_table = gr.Dataframe(
                              headers=["Metric", "Current Audit", "Industry Benchmark", "Status"],
                              value=[]
                          )


          # Bottom: GLOBAL STATUS BAR (Visible across all tabs)
          op_status_bar = gr.Markdown(
                value="**Status:** Ready | **Documents:** 0 | **Chunks:** 0 | **Cache Hits:** 0/0",
                elem_id="op_status_bar"
          )

# ----------------------------------------------- COMPONENTS WIRING (Defined with chat_interface) ------------------------------------------------------------------------------------- #

      # Chat Event handlers
      def update_status_bar():
            """Update the status bar with current statistics."""
            if doc_store.is_ready:
                stats = doc_store.processing_stats
                cache_rate = 0
                if hasattr(doc_store.retriever, 'total_queries') and doc_store.retriever.total_queries > 0:
                    cache_rate = (doc_store.retriever.cache_hits / doc_store.retriever.total_queries) * 100

                return f"**Status:** ✅ Ready | **Documents:** {stats.get('documents_found', 0)} | **Chunks:** {stats.get('total_chunks', 0)} | **Cache Rate:** {cache_rate:.0f}%"
            return "**Status:** Ready | **Documents:** 0 | **Chunks:** 0 | **Cache Hits:** 0/0"



      def clear_all():
          """Clear everything and reset the interface."""
          global doc_store, audit_logs
          doc_store = EnhancedDocumentStore()
          audit_logs = []

          # Return 14 values to match your specific UI layout
          return (
              [],                                 # 1. chatbot
              None,                               # 2. file_upload
              "",                                 # 3. chat_input
              None,                               # 4. doc_viewer (Now gr.Image, so return None)
              "",                                 # 5. structure_output_textbox
              "",                                 # 6. structure_output_code
              "",                                 # 7. extra status
              gr.update(choices=[], value=None),  # 8. doc_type_filter
              gr.update(choices=[], value=None),  # 9. view_selector
              pd.DataFrame(),                     # 10. audit_table
              None,                               # 11. audit_download_btn
              "🔄 System Reset",                  # 12. op_status_bar
              {"current_page": 0, "images": []}, # 13. viewer_state (Reset State)
              "**Page 0 of 0**"                  # 14. page_indicator (Reset Markdown)
          )


      def process_pdf_with_status(file_list):
            """Processes uploaded file and ensures UI doesn't hang on error."""
            try:
                # Calls your existing handler from Section 11;
                status, structure_json_string, structure_display, doc_type_filter, view_selector, filename_summary = process_pdf_handler(file_list)

                # UI Gloabl Status Bar
                status_bar_text = update_status_bar()

                return status, structure_json_string, structure_display, view_selector, doc_type_filter, status_bar_text, filename_summary

                return (
                    status,                     # -> status_output
                    structure_json_string,      # -> structure_output_code
                    structure_display,          # -> structure_output_textbox
                    gr.update(choices=filter_choices, value="All"), # For search filter
                    gr.update(choices=file_paths, value=file_paths[0] if file_paths else None), # For viewer
                    view_selector,
                    f"✅ {len(all_filenames)} Files Ready"
                )

            except Exception as e:
                # Debugging print to see exactly what happened in Colab logs
                print(f"Error in wrapper: {str(e)}")
                return f"❌ System Error: {str(e)}","[]", "⚠️ Error", gr.update(choices=["All"]), "Error", gr.update(choices=[])


      # UI Buttons: SUMMARY, FIND AMOUNTS,CLEAR CHAT
      # Define Example question handlers
      def ask_summary(history, doc_type_filter, auto_route, audit_num_chunks):
          """Specific wrapper for the Summary button."""

          msg = "Can you provide a summary of the main points in this document?"

          if history is None or (len(history) > 0 and not isinstance(history[0], dict)):
            history = []

          # We loop through the generator to get the final yielded history
          final_history = history

          # Drains the generator from chat_with_status
          for updated_history, status in chat_with_status(msg, history, doc_type_filter, auto_route, audit_num_chunks):
              final_history = updated_history

          return final_history


      def ask_amounts(history, doc_type_filter, auto_route, audit_num_chunks):
          """Specific wrapper for the Find Amounts button."""

          msg = "What are all the monetary amounts or financial figures mentioned?"

          if history is None or (len(history) > 0 and not isinstance(history[0], dict)):
            history = []

          final_history = history
          for updated_history, status in chat_with_status(msg, history, doc_type_filter, auto_route, audit_num_chunks):
              final_history = updated_history

          return final_history


      # --- EVENT WIRING ---

      # 🔗 1. LLM Selector
      # Connect the selector to your handle_model_Transition (UI Manager) function
      llm_selector.change(
          fn=handle_model_transition,
          inputs=[llm_selector, chatbot, clear_on_switch_checkbox],
          outputs=[chatbot, engine_status]
      )


      # 🔗 2. Processing Events

      # A. File Upload. Ensures the loading state for uploading a file is properly cleared
      #    File Preview (2 Outputs)
      # When files are picked, show the first one in the viewer and names in the debug box
      file_upload.change(
            fn=lambda x: (x[0].name if x else None, f"📑 {len(x)} files selected" if x else "No files"),
            inputs=[file_upload],
            outputs=[doc_viewer, filename_debug_output]
        )

      # B. "View File" dropdown to switch which PDF is showing in the doc_viewer
      view_selector.change(
         fn=load_pdf_into_viewer,
        inputs=[view_selector],
        outputs=[doc_viewer, viewer_state, page_indicator]
      )


      # C. Document Processing (Backend Ingestion)
      ingest_btn.click(
            fn=process_pdf_handler,
            inputs=[file_upload], # Pull from the actual uploaded file
            outputs=[
                status_output,              # Receives status_msg
                structure_output_code,      # Receives structure_json_string
                structure_output_textbox,   # Receives structure_display (Bulleted String)
                doc_type_filter,               # Receives filter update - doc_type_filter (Dropdown - Filter Document Type)
                view_selector,
                op_status_bar]              # Receives status bar update - status_bar_text (String - Global Status Indicator)
        )


      # D. Document Viewer Navigation: Previous Button
      prev_btn.click(
          fn=flip_page,
          inputs=[gr.State("prev"), viewer_state],
          outputs=[doc_viewer, viewer_state, page_indicator]
      )

      # E. Document Viewer Navigation: Next Button
      next_btn.click(
          fn=flip_page,
          inputs=[gr.State("next"), viewer_state],
           outputs=[doc_viewer, viewer_state, page_indicator]
      )


      # 🔗 3. Chat Texbox (Message) Input & Send

      # A. Chat Functionality (The "Conversation" bridge)
      # Use .then() to clear the input box after sending
      msg_input.submit(
            fn=chat_with_status,
            inputs=[msg_input, chatbot, doc_type_filter, auto_route, audit_num_chunks],
            outputs=[chatbot, op_status_bar]
      ).then(lambda: "", None, [msg_input]) # Only clear the input


      # B. Chat Functionality (The "Conversation" bridge)
      # Use .then() to clear the input box after sending
      send_btn.click(
            fn=chat_with_status,
            inputs=[msg_input, chatbot, doc_type_filter, auto_route, audit_num_chunks],
            outputs=[chatbot, op_status_bar]
      ).then(lambda: "", None, [msg_input])



      # 🔗 4. Download Chat History & Performance Audit Events

      # A. When the button is clicked:
      # -----Take 'chatbot' as input, run 'export_chat_history_to_pdf',
      # -----and send the result to 'chat_download_file'
      chat_download_btn.click(
          fn=export_chat_history_to_pdf,
          inputs=[chatbot],
          outputs=[chat_download_btn]
      )

      # B. Run Performance Audit
      run_audit_btn.click(
          fn=run_performance_audit,
          inputs=[sector_dropdown, audit_num_chunks],
          outputs=[latency_stat, audit_accuracy_gauge, accuracy_gauge, bottleneck_plot, audit_table]
      )


      # B. Export & Download Performance Audit
      audit_download_btn.click(
          fn=handle_audit_export_ui,
          inputs=[audit_table],
          outputs=[
              audit_download_btn,  # Receives the file update
              op_status_bar         # Receives the status message (the "✅ Audit report..." text)
          ]
      )


      # 🔗 5. UI Utility Events

      # A. Utility/Reset Buttons: Clear ALL (Start new)
      # Clear the entire platform
      clear_all_btn.click(
            fn=clear_all,
            inputs=[],
            outputs=[
                chatbot,
                file_upload,
                status_output,
                doc_viewer,
                filename_debug_output,
                structure_output_code,
                structure_output_textbox,
                doc_type_filter,  # Ensure this matches the name in your UI layout
                view_selector,
                audit_table,
                audit_download_file,
                op_status_bar
           ]
     )

      # B. Utility/Reset Buttons: Clear Chat History
      clear_chat_btn.click(
          fn=lambda: (
              [{"role": "assistant", "content": "**🤖 AI Document Assistant:** 👋 Chat cleared. How can I help you with your documents? 🚀"}],
              gr.update(visible=False)
          ),
          inputs=None,
          outputs=[chatbot, chat_download_file]
      )

      # C. Summary Button Wiring
      example_btn1.click(
          fn=ask_summary,
          inputs=[chatbot, doc_type_filter, auto_route, audit_num_chunks],
          outputs=[chatbot]
      )

      # D. Find Amounts Button Wiring
      example_btn2.click(
          fn=ask_amounts,
          inputs=[chatbot, doc_type_filter, auto_route, audit_num_chunks],
          outputs=[chatbot]
      )

      return demo

      # 🔗 6. ADDED - Initialize JavaScript for Auto-Scroll
      demo.load(js=scroll_script)

print("✅ SSECTION 12. GRADIO INTERFACE, CHAT HANDLERS, & WIRING LOGIC Complete.")



✅ SSECTION 12. GRADIO INTERFACE, CHAT HANDLERS, & WIRING LOGIC Complete.


# **SECTION 13. APPLICATION LAUNCHER**

**Logic and Flow Analysis**

Section 13 is the **Ignition System.** This is the final step that transitions the code from a collection of functions and classes into a live, interactive web service.

<br>

The launcher performs several critical operational tasks:

1. **Port Management:** `gr.close_all()` ensures that any previous instances of the app running in the background are terminated, preventing "Address already in use" errors—a common headache in development environments like Google Colab or Jupyter.

2. **Theme Injection:** It applies the `gr.themes.Soft()` base and overlays your `custom_css`. This creates the specific "Dark/Obsidian" professional aesthetic you designed in Section 12.

3. **Tunneling:** By setting `share=True`, Gradio creates a secure public URL (e.g., `https://xyz123.gradio.live`). This allows you to test the mobile responsiveness of your platform or share the MVP with stakeholders without deploying to a cloud server.

4. **Debugging:** `debug=True` is vital for the MVP stage. If the AI fails to process a specific PDF, the error logs will print directly in your notebook cell, allowing for immediate troubleshooting.

In [ ]:
# ------- SECTION 13. APPLICATION LAUNCHER -------

# 1. Cleanup: Close any existing Gradio servers to free up ports
gr.close_all()

print("🚀 Initializing Platform Components...")
print("📂 Loading Vector Store...")
print("🧠 Connecting LLM Engine...")


# 2. Build the Interface
# Calls the create_interface() function defined in Section 12
demo = create_interface()

if __name__ == "__main__":
    print("🚀 AI-Powered Document Intelligence Automation Platform Launching...")

    demo.launch(
        theme=gr.themes.Soft(),
        css=custom_css,
        debug=True,
        share=True
    )

🚀 Initializing Platform Components...
📂 Loading Vector Store...
🧠 Connecting LLM Engine...
🚀 AI-Powered Document Intelligence Automation Platform Launching...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b85cb655aef2dbb370.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🔄 Switching to Mistral 7B...


llama_context: n_ctx_per_seq (16384) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


✅ Mistral 7B API Key Loaded & Configured.
📖 Starting PDF extraction and analysis for: {original_filename}
